In [1]:
import arcpy
import pandas as pd
import numpy as np
from collections import namedtuple, defaultdict
from pathlib import Path

In [2]:
counties_dbf = r"C:\gis\Projects\HistoricCounties\Data\HistoricalCountyBoundaries\UT_Historical_Counties\UT_Historical_Counties.dbf"
districts_csv = r"C:\gis\Projects\HistoricCounties\Data\JudicialDistricts\test.csv"
counties_shp = r"C:\gis\Projects\HistoricCounties\Data\HistoricalCountyBoundaries\UT_Historical_Counties\UT_Historical_Counties.shp"


# Initial Stuff

In [3]:
counties_list = []
county_fields = [f.name for f in arcpy.ListFields(counties_shp)]
county_fields.append('SHAPE@')
with arcpy.da.SearchCursor(counties_shp, county_fields) as sc:
    for row in sc:
        counties_list.append(dict(zip(county_fields, row)))
    

In [4]:
counties_df = pd.DataFrame(counties_list)

In [5]:
counties_df.columns

Index(['FID', 'Shape', 'NAME', 'ID', 'STATE', 'FIPS', 'VERSION', 'START_DATE',
       'END_DATE', 'CHANGE', 'CITATION', 'START_N', 'END_N', 'AREA_SQMI',
       'DATASET', 'CNTY_TYPE', 'FULL_NAME', 'SHAPE@'],
      dtype='object')

In [94]:
counties_df['START_DATE']

0     1849-07-02
1     1850-10-05
2     1850-12-03
3     1850-10-05
4     1850-10-05
         ...    
208   1921-03-10
209   1931-03-23
210   1931-03-23
211   2003-01-01
212   2003-01-01
Name: START_DATE, Length: 213, dtype: datetime64[ns]

In [20]:
districts_df = pd.read_csv(r"C:\gis\Projects\HistoricCounties\Data\JudicialDistricts\test.csv")
districts_df

,CountyName,StartDate,EndDate,NewDistrict,OldDistrict,Version
0,Beaver,1856-01-17,1859-01-20,2,2,1
1,Beaver,1859-01-21,1862-01-16,1,2,2
2,Beaver,1862-01-17,1896-01-05,2,1,3
3,Beaver,1896-01-06,2030-01-01,5,2,4
4,Box Elder,1856-01-17,1859-01-20,1,1,1
...,...,...,...,...,...,...
113,Richland,1865-12-27,NaN,3,3,1
114,Rio Virgen,1869-02-18,NaN,2,2,1
115,St. Marys,1856-01-17,1859-01-20,3,3,1
116,St. Marys,1859-01-21,NaN,2,3,2


In [21]:
districts_df['StartDate'] = pd.to_datetime(districts_df['StartDate'])
districts_df['EndDate'] = pd.to_datetime(districts_df['EndDate'])
districts_df['district_key']=districts_df['CountyName'].str.casefold() + '_D' + districts_df['Version'].astype(str)  #: add district key
districts_df['StartDate']


0     1856-01-17
1     1859-01-21
2     1862-01-17
3     1896-01-06
4     1856-01-17
         ...    
113   1865-12-27
114   1869-02-18
115   1856-01-17
116   1859-01-21
117   1856-01-17
Name: StartDate, Length: 118, dtype: datetime64[ns]

In [13]:
districts_df.dtypes

CountyName              object
StartDate       datetime64[ns]
EndDate         datetime64[ns]
NewDistrict             object
OldDistrict             object
Version                float64
district_key            object
dtype: object

In [7]:
counties_df['county_key']=counties_df['ID'] + '_S' + counties_df['VERSION'].astype(str)

In [15]:
counties_df[counties_df['ID'].str.contains('beaver')]

,FID,Shape,NAME,ID,STATE,FIPS,VERSION,START_DATE,END_DATE,CHANGE,CITATION,START_N,END_N,AREA_SQMI,DATASET,CNTY_TYPE,FULL_NAME,county_key
56,56,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,UT,49001,1,1856-01-05,1861-01-17,BEAVER created by Utah Territory from IRON; BE...,"(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)",18560105,18610117,15141.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S1
68,68,"(-112.03271613528274, 38.325452153781306)",BEAVER,uts_beaver,UT,49001,3,1861-02-28,1862-01-16,The United States created Colorado Territory f...,"(U.S. Stat., vol. 12, ch. 59[1861]/pp. 172-177...",18610228,18620116,7803.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S3
78,78,"(-110.7025292913094, 38.32662877359086)",BEAVER,uts_beaver,UT,49001,2,1861-01-18,1861-02-27,"BEAVER lost to HUMBOLDT (Utah Territory, extin...","(Utah Terr. Laws 1861, 10th sess., p. 19)",18610118,18610227,11279.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S2
86,86,"(-112.0228053482935, 38.363273470514734)",BEAVER,uts_beaver,UT,49001,5,1862-07-14,1865-01-15,BEAVER lost to Nevada Territory in present Nev...,"(U.S. Stat., vol. 12, ch. 173 [1862], p. 575; ...",18620714,18650115,9483.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S5
100,100,"(-112.5151993561241, 38.36318198233733)",BEAVER,uts_beaver,UT,49001,4,1862-01-17,1862-07-13,"BEAVER gained from MILLARD, SANPETE, and ST. M...","(Utah Terr. Laws 1861, 11th sess., sec. 3/p. 47)",18620117,18620713,11049.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S4
120,120,"(-113.70827225556754, 38.3589546264961)",BEAVER,uts_beaver,UT,49001,6,1865-01-16,1866-05-04,BEAVER lost to creation of PIUTE.,"(Utah Terr. Laws 1864, 14th sess., p. 16)",18650116,18660504,4094.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S6
124,124,"(-113.23575592353453, 38.35770859461383)",BEAVER,uts_beaver,UT,49001,7,1866-05-05,2003-12-31,BEAVER lost to the state of Nevada when Nevada...,"(U.S. Stat., vol. 14, ch. 73[1866]/p. 43; Van ...",18660505,20031231,2592.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S7


In [16]:
class County:
    def __init__(self, name):
        self.name = name.casefold()
        self.shape_df = None  #: All entries for this county in the historical county shape/geography dataframe
        self.district_df = None  #: All entries for this county in the historical dsitrict composition dataframe
        self.all_dates = None  #: Series of start dates day from both shape_df and district_df
        
    def setup(self, counties_df, districts_df):
        #: create a local shape dataframe of just the county, change the index to the start date of each version
        self.shape_df = counties_df[counties_df['ID'].str.contains(self.name)].copy()  #: copy to avoid chained indexing
        self.shape_df.set_index('START_DATE', inplace=True)
        self.shape_df.sort_index(inplace=True)

        #: create a local district dataframe of just the county, change the index to the start date of each version
        self.district_df = districts_df[districts_df['CountyName'].str.contains(self.name, case=False)].copy()
        self.district_df.set_index('StartDate', inplace=True)
        self.district_df.sort_index(inplace=True)

        #: create series of dates to check 
        self.all_dates = pd.Series(self.shape_df.index.union(self.district_df.index))
        
        
    def get_versions(self):
#         check_dates = self.all_dates + pd.Timedelta(days=1)
#         versions_df = pd.DataFrame(self.all_dates + pd.Timedelta(days=1))
        pass
        

In [17]:
class ChangeDate:
    def __init__(self, date):
        self.date = date
        self.county_version = 'n/a'
        self.district = 'n/a'
        
    def __repr__(self):
        out_string = ', '.join([str(self.date), self.county_version, str(self.district)])
        return out_string

In [18]:
beaver = County('Beaver')
beaver.setup(counties_df, districts_df)

dates = list(beaver.shape_df.index.union(beaver.district_df.index))
change_dates = [ChangeDate(date) for date in dates]
for change_date in change_dates:
    for county_row in beaver.shape_df.itertuples():
        if change_date.date >= county_row.Index and change_date.date <= county_row.END_DATE:
            change_date.county_version = county_row.county_key
            break
    for district_row in beaver.district_df.itertuples():
        if change_date.date >= district_row.Index and change_date.date <= district_row.EndDate:
            change_date.district = district_row.NewDistrict
            break
change_dates

#: Have to convert to namedtuple afterwards so that we can alter the list of ChangeDate items as we iterate
ChangeDatesTuple = namedtuple('ChangeDatesTuple', 'date county_version district')
new_dates = [ChangeDatesTuple(cd.date, cd.county_version, cd.district) for cd in change_dates]
new_dates
dates_df = pd.DataFrame(new_dates)
dates_df
    

,date,county_version,district
0,1856-01-05,uts_beaver_S1,n/a
1,1856-01-17,uts_beaver_S1,2
2,1859-01-21,uts_beaver_S1,1
3,1861-01-18,uts_beaver_S2,1
4,1861-02-28,uts_beaver_S3,1
5,1862-01-17,uts_beaver_S4,2
6,1862-07-14,uts_beaver_S5,2
7,1865-01-16,uts_beaver_S6,2
8,1866-05-05,uts_beaver_S7,2
9,1896-01-06,uts_beaver_S7,5


In [54]:
all_times + pd.Timedelta(days=1)

0   1856-01-06
1   1856-01-18
2   1859-01-22
3   1861-01-19
4   1861-03-01
5   1862-01-18
6   1862-07-15
7   1865-01-17
8   1866-05-06
9   1896-01-07
dtype: datetime64[ns]

In [19]:
dfs_by_county = {}
for county in counties_df['ID'].unique():
    county_name = county.split('_')[-1]
    dfs_by_county[county_name] = counties_df[counties_df['ID'] == county]

In [20]:
dfs_by_county

{'des':     FID                                     Shape     NAME   ID STATE FIPS  \
 0     0  (-113.45238337501421, 38.03305439561905)  Deseret  des    UT        
 5     5  (-113.48001976528437, 38.00695181641287)  Deseret  des              
 13   13  (-113.47478944961205, 38.00598435115826)  Deseret  des              
 
     VERSION START_DATE   END_DATE  \
 0         1 1849-07-02 1850-01-30   
 5         3 1850-12-03 1851-04-04   
 13        2 1850-01-31 1850-12-02   
 
                                                CHANGE  \
 0   The General Assembly of the newly proposed sta...   
 5   The state of Deseret lost non-county area to I...   
 13  The state of Deseret lost non-county area to c...   
 
                                              CITATION   START_N     END_N  \
 0   (Atlas of Utah, 160-161; Brown, Cannon, and Ja...  18490702  18500130   
 5                                (Morgan, 56-57, 157)  18501203  18510404   
 13                              (Morgan, pp. 180-181

In [25]:
for c in counties_df['ID'].unique():
    print(c)

des
uts_saltlake
uts_iron
uts_davis
uts_weber
uts_utah
uts_sanpete
uts_tooele
utt
uts_millard
uts_washington
uts_juab
utt_desert
utt_greenriver
uts_summit
utt_carson
utt_shambip
utt_cedar
uts_cache
uts_boxelder
utt_malad
uts_beaver
utt_humboldt
utt_greasewood
utt_stmarys
jet_mountain
jet_saratoga
jet_part
uts_morgan
uts_wasatch
uts_kane
uts_rich
uts_uintah
uts_sevier
uts_piute
utt_riovirgen
uts_sanjuan
uts_emery
uts_garfield
uts_grand
uts_wayne
uts_carbon
uts_duchesne
uts_daggett


In [37]:
districts_df['CountyName'].unique()

array(['Beaver', 'Box Elder', 'Cache', 'Carbon', 'Daggett', 'Davis',
       'Duchesne', 'Emery', 'Garfield', 'Grand', 'Iron', 'Juab', 'Kane',
       'Millard', 'Morgan', 'Piute', 'Rich', 'Salt Lake', 'San Juan',
       'Sanpete', 'Sevier', 'Summit', 'Tooele', 'Uintah', 'Utah',
       'Wasatch', 'Washington', 'Wayne', 'Weber', 'Carson', 'Cedar',
       'Desert', 'Greasewood', 'Green River', 'Humboldt', 'Malad',
       'Richland', 'Rio Virgen', 'St. Marys', 'Shambip'], dtype=object)

In [28]:
def clean_county_name(name):
    clean_name = name.casefold()
    clean_name = clean_name.replace(' ', '')
    clean_name = clean_name.replace('.', '')
#     if clean_name == 'richland':  #: custom to handle Richland renamed to Rich
#         clean_name = 'rich'
        
    return clean_name

In [33]:
for county in districts_df['CountyName'].unique():
    county_key = clean_county_name(county)
    if county_key in dfs_by_county:
        print(dfs_by_county[county_key]['county_key'])

56     uts_beaver_S1
68     uts_beaver_S3
78     uts_beaver_S2
86     uts_beaver_S5
100    uts_beaver_S4
120    uts_beaver_S6
124    uts_beaver_S7
Name: county_key, dtype: object
52     uts_boxelder_S1
88     uts_boxelder_S3
105    uts_boxelder_S2
126    uts_boxelder_S4
158    uts_boxelder_S5
Name: county_key, dtype: object
49     uts_cache_S1
99     uts_cache_S2
107    uts_cache_S3
Name: county_key, dtype: object
186    uts_carbon_S1
Name: county_key, dtype: object
199    uts_daggett_S1
200    uts_daggett_S2
Name: county_key, dtype: object
3      uts_davis_S1
27     uts_davis_S2
39     uts_davis_S3
44     uts_davis_S4
93     uts_davis_S5
159    uts_davis_S6
Name: county_key, dtype: object
191    uts_duchesne_S1
194    uts_duchesne_S2
Name: county_key, dtype: object
153    uts_emery_S2
162    uts_emery_S1
176    uts_emery_S3
187    uts_emery_S4
211    uts_emery_S5
Name: county_key, dtype: object
164    uts_garfield_S1
168    uts_garfield_S2
Name: county_key, dtype: object
177    uts_gr

In [ ]:
def in_date_range(start_date, end_date):
    if 

In [ ]:
for county_name in districts_df['CountyName'].unique():
    county_dict_key = county_name.casefold().replace(' ', '') 
    if county_dict_key in dfs_by_county:
        for df in dfs_by_county[county_dict_key]:

In [40]:
'des'.split('_')

['des']

In [42]:
'ut_saltlake'.split('_')

['ut', 'saltlake']

In [50]:
'salt lake'.replace(' ', '')

'saltlake'

In [2]:
test_df = pd.DataFrame(data={'county_key':['uts_co_S1', 'uts_co_s2', 'uts_co_s3'], 'END_DATE':['2020-02-19', '2020-02-24', '2020-02-28']}, index=['2020-02-05', '2020-02-20', '2020-02-25'])
test_df.index = pd.to_datetime(test_df.index)
test_df['END_DATE'] = pd.to_datetime(test_df['END_DATE'])
test_df.index

DatetimeIndex(['2020-02-05', '2020-02-20', '2020-02-25'], dtype='datetime64[ns]', freq=None)

In [3]:
test_df

,county_key,END_DATE
2020-02-05,uts_co_S1,2020-02-19
2020-02-20,uts_co_s2,2020-02-24
2020-02-25,uts_co_s3,2020-02-28


In [4]:
shape_df = pd.DataFrame(
    data={
        'county_key': ['uts_co_S1', 'uts_co_s2', 'uts_co_s3'],
        'END_DATE': ['2020-02-19', '2020-02-24', '2020-02-28'],
    },
    index=['2020-02-05', '2020-02-20', '2020-02-25']
)
shape_df.index = pd.to_datetime(shape_df.index)
shape_df['END_DATE'] = pd.to_datetime(shape_df['END_DATE'])

districts_df = pd.DataFrame(
    data={
        'NewDistrict': ['1', '2', '3'],
        'EndDate': ['2020-02-14', '2020-02-24', '2020-02-28'],
    },
    index=['2020-02-10', '2020-02-15', '2020-02-25']
)
districts_df.index = pd.to_datetime(districts_df.index)
districts_df['EndDate'] = pd.to_datetime(districts_df['EndDate'])

In [6]:
districts_df

,NewDistrict,EndDate
2020-02-10,1,2020-02-14
2020-02-15,2,2020-02-24
2020-02-25,3,2020-02-28


In [8]:
pd.Series(shape_df.index.union(districts_df.index))

0   2020-02-05
1   2020-02-10
2   2020-02-15
3   2020-02-20
4   2020-02-25
dtype: datetime64[ns]

In [14]:
districts_df['OldDistrict'] = ['10', '1', '2']

In [15]:
districts_df

,NewDistrict,EndDate,OldDistrict
2020-02-10,1,2020-02-14,10
2020-02-15,2,2020-02-24,1
2020-02-25,3,2020-02-28,2


In [5]:
districts_df.OldDistrict.eq(districts_df.NewDistrict.shift())

0      False
1       True
2       True
3       True
4      False
       ...  
113     True
114    False
115    False
116     True
117    False
Length: 118, dtype: bool

In [6]:
districts_df.NewDistrict.shift()

0      NaN
1        2
2        1
3        2
4        5
      ... 
113      3
114      3
115      2
116      3
117      2
Name: NewDistrict, Length: 118, dtype: object

In [12]:
# districts_df.set_index('StartDate', inplace=True)
districts_df.sort_index(inplace=True)

districts_df.index.eq(districts_df['EndDate'].shift() + pd.Timedelta(days=1))

TypeError: 'DatetimeIndex' object is not callable

In [18]:
districts_df.loc[:, ['CountyName', 'EndDate']]

,CountyName,EndDate
StartDate,,
1851-01-08,Salt Lake,1856-01-16
1851-01-08,Tooele,1856-01-16
1851-01-08,Utah,1851-10-03
1851-08-08,Iron,1855-01-18
1851-08-08,Davis,1851-10-03
...,...,...
1982-02-16,Millard,2030-01-01
1982-02-16,Garfield,2030-01-01
1988-04-25,Duchesne,2030-01-01


In [22]:
change_csv_path = r"C:\gis\Projects\HistoricCounties\Data\JudicialDistricts\change_dates.csv"
change_df = pd.read_csv(change_csv_path)

In [23]:
change_df

,Unnamed: 0,date,county_name,county_version,district,end_date
0,0,1856-01-05,beaver,uts_beaver_S1,NaN,1856-01-16
1,1,1856-01-17,beaver,uts_beaver_S1,2,1859-01-20
2,2,1859-01-21,beaver,uts_beaver_S1,1,1861-01-17
3,3,1861-01-18,beaver,uts_beaver_S2,1,1861-02-27
4,4,1861-02-28,beaver,uts_beaver_S3,1,1862-01-16
...,...,...,...,...,...,...
304,2,1859-01-21,stmarys,utt_stmarys_S1,NaN,1861-01-17
305,3,1861-01-18,stmarys,utt_stmarys_S2,NaN,NaN
306,0,1856-01-12,shambip,utt_shambip_S1,NaN,1856-01-16
307,1,1856-01-17,shambip,utt_shambip_S1,NaN,1861-01-17


In [10]:
sorted(change_df['date'].unique())
counts = change_df['date'].value_counts()
type(counts)

pandas.core.series.Series

In [11]:
display(counts.sort_index())

1850-01-31    6
1850-10-05    3
1850-12-03    1
1851-01-08    3
1851-08-08    4
             ..
1975-03-21    1
1976-02-10    1
1982-02-16    2
1988-04-25    3
2003-01-01    2
Name: date, Length: 69, dtype: int64

In [12]:
display(counties_df[counties_df['county_key'] == 'uts_beaver_S1'])

,FID,Shape,NAME,ID,STATE,FIPS,VERSION,START_DATE,END_DATE,CHANGE,CITATION,START_N,END_N,AREA_SQMI,DATASET,CNTY_TYPE,FULL_NAME,SHAPE@,county_key
56,56,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,UT,49001,1,1856-01-05,1861-01-17,BEAVER created by Utah Territory from IRON; BE...,"(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)",18560105,18610117,15141.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S1


In [26]:
test_combined_keys_df = counties_df[counties_df['ID'].str.contains('beaver')].copy()
test_district_keys_df = districts_df[districts_df['CountyName'].str.contains('beaver', case=False)].copy()
test_combined_keys_df['DISTRICT'] = np.nan
display(test_district_keys_df)

,CountyName,StartDate,EndDate,NewDistrict,OldDistrict,Version,district_key
0,Beaver,1856-01-17,1859-01-20,2,2,1,beaver_D1
1,Beaver,1859-01-21,1862-01-16,1,2,2,beaver_D2
2,Beaver,1862-01-17,1896-01-05,2,1,3,beaver_D3
3,Beaver,1896-01-06,2030-01-01,5,2,4,beaver_D4


In [14]:
# test_combined_keys_df[test_combined_keys_df['county_key'] == 'uts_beaver_S1']
# test_combined_keys_df.sort_values(by='START_DATE', inplace=True)
# display(test_combined_keys_df)

test_df = change_df[change_df['county_name'] == 'beaver'].copy()
display(test_df)

,Unnamed: 0,date,county_name,county_version,district,end_date
0,0,1856-01-05,beaver,uts_beaver_S1,NaN,1856-01-16
1,1,1856-01-17,beaver,uts_beaver_S1,2,1859-01-20
2,2,1859-01-21,beaver,uts_beaver_S1,1,1861-01-17
3,3,1861-01-18,beaver,uts_beaver_S2,1,1861-02-27
4,4,1861-02-28,beaver,uts_beaver_S3,1,1862-01-16
5,5,1862-01-17,beaver,uts_beaver_S4,2,1862-07-13
6,6,1862-07-14,beaver,uts_beaver_S5,2,1865-01-15
7,7,1865-01-16,beaver,uts_beaver_S6,2,1866-05-04
8,8,1866-05-05,beaver,uts_beaver_S7,2,1896-01-05
9,9,1896-01-06,beaver,uts_beaver_S7,5,NaN


In [15]:
merged_df = pd.merge(test_df, test_combined_keys_df, left_on='county_version', right_on='county_key', validate='m:1')
district_merged_df = pd.merge(merged_df, test_combined_keys_df, left_on='county_version', right_on='county_key', validate='m:1')
display(merged_df)

,Unnamed: 0,date,county_name,county_version,district,end_date,FID,Shape,NAME,ID,...,CITATION,START_N,END_N,AREA_SQMI,DATASET,CNTY_TYPE,FULL_NAME,SHAPE@,county_key,DISTRICT
0,0,1856-01-05,beaver,uts_beaver_S1,NaN,1856-01-16,56,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,...,"(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)",18560105,18610117,15141.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S1,NaN
1,1,1856-01-17,beaver,uts_beaver_S1,2,1859-01-20,56,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,...,"(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)",18560105,18610117,15141.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S1,NaN
2,2,1859-01-21,beaver,uts_beaver_S1,1,1861-01-17,56,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,...,"(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)",18560105,18610117,15141.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S1,NaN
3,3,1861-01-18,beaver,uts_beaver_S2,1,1861-02-27,78,"(-110.7025292913094, 38.32662877359086)",BEAVER,uts_beaver,...,"(Utah Terr. Laws 1861, 10th sess., p. 19)",18610118,18610227,11279.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S2,NaN
4,4,1861-02-28,beaver,uts_beaver_S3,1,1862-01-16,68,"(-112.03271613528274, 38.325452153781306)",BEAVER,uts_beaver,...,"(U.S. Stat., vol. 12, ch. 59[1861]/pp. 172-177...",18610228,18620116,7803.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S3,NaN
5,5,1862-01-17,beaver,uts_beaver_S4,2,1862-07-13,100,"(-112.5151993561241, 38.36318198233733)",BEAVER,uts_beaver,...,"(Utah Terr. Laws 1861, 11th sess., sec. 3/p. 47)",18620117,18620713,11049.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S4,NaN
6,6,1862-07-14,beaver,uts_beaver_S5,2,1865-01-15,86,"(-112.0228053482935, 38.363273470514734)",BEAVER,uts_beaver,...,"(U.S. Stat., vol. 12, ch. 173 [1862], p. 575; ...",18620714,18650115,9483.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S5,NaN
7,7,1865-01-16,beaver,uts_beaver_S6,2,1866-05-04,120,"(-113.70827225556754, 38.3589546264961)",BEAVER,uts_beaver,...,"(Utah Terr. Laws 1864, 14th sess., p. 16)",18650116,18660504,4094.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S6,NaN
8,8,1866-05-05,beaver,uts_beaver_S7,2,1896-01-05,124,"(-113.23575592353453, 38.35770859461383)",BEAVER,uts_beaver,...,"(U.S. Stat., vol. 14, ch. 73[1866]/p. 43; Van ...",18660505,20031231,2592.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S7,NaN
9,9,1896-01-06,beaver,uts_beaver_S7,5,NaN,124,"(-113.23575592353453, 38.35770859461383)",BEAVER,uts_beaver,...,"(U.S. Stat., vol. 14, ch. 73[1866]/p. 43; Van ...",18660505,20031231,2592.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S7,NaN


In [19]:
df_renamer = {
    'NAME': 'NAME',
    'ID': 'ID',
    'FIPS': 'FIPS',
    'VERSION': 'SHP_VERSION',
    'START_DATE': 'SHP_START_DATE',
    'END_DATE': 'SHP_END_DATE',
    'CHANGE': 'SHP_CHANGE',
    'CITATION': 'SHP_CITATION',
    'AREA_SQMI': 'SHP_AREA_SQMI',
    'date': 'DST_START_DATE',
    'end_date': 'DST_END_DATE',
    'district': 'DST_NUMBER',
    'county_key': 'COUNTY_KEY',
}

renamed_df = merged_df.rename(columns=df_renamer)
# out_cols = df_renamer.values()
# out_cols
renamed_df[df_renamer.values()]

,NAME,ID,FIPS,SHP_VERSION,SHP_START_DATE,SHP_END_DATE,SHP_CHANGE,SHP_CITATION,SHP_AREA_SQMI,DST_START_DATE,DST_END_DATE,DST_NUMBER,COUNTY_KEY
0,BEAVER,uts_beaver,49001,1,1856-01-05,1861-01-17,BEAVER created by Utah Territory from IRON; BE...,"(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)",15141.0,1856-01-05,1856-01-16,NaN,uts_beaver_S1
1,BEAVER,uts_beaver,49001,1,1856-01-05,1861-01-17,BEAVER created by Utah Territory from IRON; BE...,"(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)",15141.0,1856-01-17,1859-01-20,2,uts_beaver_S1
2,BEAVER,uts_beaver,49001,1,1856-01-05,1861-01-17,BEAVER created by Utah Territory from IRON; BE...,"(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)",15141.0,1859-01-21,1861-01-17,1,uts_beaver_S1
3,BEAVER,uts_beaver,49001,2,1861-01-18,1861-02-27,"BEAVER lost to HUMBOLDT (Utah Territory, extin...","(Utah Terr. Laws 1861, 10th sess., p. 19)",11279.0,1861-01-18,1861-02-27,1,uts_beaver_S2
4,BEAVER,uts_beaver,49001,3,1861-02-28,1862-01-16,The United States created Colorado Territory f...,"(U.S. Stat., vol. 12, ch. 59[1861]/pp. 172-177...",7803.0,1861-02-28,1862-01-16,1,uts_beaver_S3
5,BEAVER,uts_beaver,49001,4,1862-01-17,1862-07-13,"BEAVER gained from MILLARD, SANPETE, and ST. M...","(Utah Terr. Laws 1861, 11th sess., sec. 3/p. 47)",11049.0,1862-01-17,1862-07-13,2,uts_beaver_S4
6,BEAVER,uts_beaver,49001,5,1862-07-14,1865-01-15,BEAVER lost to Nevada Territory in present Nev...,"(U.S. Stat., vol. 12, ch. 173 [1862], p. 575; ...",9483.0,1862-07-14,1865-01-15,2,uts_beaver_S5
7,BEAVER,uts_beaver,49001,6,1865-01-16,1866-05-04,BEAVER lost to creation of PIUTE.,"(Utah Terr. Laws 1864, 14th sess., p. 16)",4094.0,1865-01-16,1866-05-04,2,uts_beaver_S6
8,BEAVER,uts_beaver,49001,7,1866-05-05,2003-12-31,BEAVER lost to the state of Nevada when Nevada...,"(U.S. Stat., vol. 14, ch. 73[1866]/p. 43; Van ...",2592.0,1866-05-05,1896-01-05,2,uts_beaver_S7
9,BEAVER,uts_beaver,49001,7,1866-05-05,2003-12-31,BEAVER lost to the state of Nevada when Nevada...,"(U.S. Stat., vol. 14, ch. 73[1866]/p. 43; Van ...",2592.0,1896-01-06,NaN,5,uts_beaver_S7


In [21]:
list(merged_df.columns)

['Unnamed: 0',
 'date',
 'county_name',
 'county_version',
 'district',
 'end_date',
 'FID',
 'Shape',
 'NAME',
 'ID',
 'STATE',
 'FIPS',
 'VERSION',
 'START_DATE',
 'END_DATE',
 'CHANGE',
 'CITATION',
 'START_N',
 'END_N',
 'AREA_SQMI',
 'DATASET',
 'CNTY_TYPE',
 'FULL_NAME',
 'SHAPE@',
 'county_key',
 'DISTRICT']

In [20]:
merged_df.values.tolist()

[[0,
  '1856-01-05',
  'beaver',
  'uts_beaver_S1',
  nan,
  '1856-01-16',
  56,
  (-112.17937602433766, 38.32630597762866),
  'BEAVER',
  'uts_beaver',
  'UT',
  '49001',
  1,
  Timestamp('1856-01-05 00:00:00'),
  Timestamp('1861-01-17 00:00:00'),
  'BEAVER created by Utah Territory from IRON; BEAVER included parts of present Utah, Colorado, and Nevada.',
  '(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)',
  18560105,
  18610117,
  15141.0,
  'UT_Historical_Counties',
  'County',
  'BEAVER',
  <Polygon object at 0x2e16a1a9e48[0x2e16a1a7c90]>,
  'uts_beaver_S1',
  nan],
 [1,
  '1856-01-17',
  'beaver',
  'uts_beaver_S1',
  '2',
  '1859-01-20',
  56,
  (-112.17937602433766, 38.32630597762866),
  'BEAVER',
  'uts_beaver',
  'UT',
  '49001',
  1,
  Timestamp('1856-01-05 00:00:00'),
  Timestamp('1861-01-17 00:00:00'),
  'BEAVER created by Utah Territory from IRON; BEAVER included parts of present Utah, Colorado, and Nevada.',
  '(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)',
  18560105,


In [33]:
out_path = r'C:\gis\Projects\HistoricCounties\HistoricCounties.gdb'
Path(out_path).with_suffix('.prj')

WindowsPath('C:/gis/Projects/HistoricCounties/HistoricCounties.prj')

# Datetime testing

In [4]:
test_df = pd.DataFrame(
    data = {
        'a':[1, 2, 3],
        'time':['1/2/2013', None, '2/3/2014']
    }
)
test_df['time'] = pd.to_datetime(test_df['time'])
display(test_df)

,a,time
0,1,2013-01-02
1,2,NaT
2,3,2014-02-03


In [7]:
test_df= test_df.where(pd.notnull(test_df), None)
display(test_df)

,a,time
0,1,2013-01-02
1,2,NaT
2,3,2014-02-03


In [9]:
test_df['time'].fillna(value=None)

ValueError: Must specify a fill 'value' or 'method'.

In [19]:
type(test_df.iat[1, 1])

pandas._libs.tslibs.nattype.NaTType

In [23]:
def clean_row(row):
    new_row = []
    for item in row:
        if pd.isnull(item):
            new_row.append(None)
        else:
            new_row.append(item)
    return new_row


In [24]:
for row in test_df.values.tolist():
    print(clean_row(row))

[1, Timestamp('2013-01-02 00:00:00')]
[2, None]
[3, Timestamp('2014-02-03 00:00:00')]


# Change row testing

In [2]:
rows = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
]

for row in rows:
    if row == rows[-1]:
        print(row)
        

[7, 8, 9]


# Indexing output DF

In [2]:
renamed_df = pd.read_pickle(r'C:\gis\Projects\HistoricCounties\Data\JudicialDistricts\renamed_df.pickle')

In [3]:
renamed_df

,CHANGE_DATE,CHANGE_ID,county_version,DST_NUMBER,district_version,change_end_date,FID,Shape,SHP_NAME,SHP_ID,...,SHAPE@,SHP_KEY,SHP_START_DATE,CountyName,DST_END_DATE,NewDistrict,OldDistrict,Version,DST_KEY,DST_START_DATE
0,1856-01-05,beaver,uts_beaver_S1,n/a,n/a,1856-01-16,56.0,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,...,(<geoprocessing array object object at 0x00000...,uts_beaver_S1,1856-01-05,NaN,NaT,NaN,NaN,NaN,NaN,NaT
1,1856-01-17,beaver,uts_beaver_S1,2,beaver_D1,1859-01-20,56.0,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,...,(<geoprocessing array object object at 0x00000...,uts_beaver_S1,1856-01-05,beaver,1859-01-20,2,2,1.0,beaver_D1,1856-01-17
2,1859-01-21,beaver,uts_beaver_S1,1,beaver_D2,1861-01-17,56.0,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,...,(<geoprocessing array object object at 0x00000...,uts_beaver_S1,1856-01-05,beaver,1862-01-16,1,2,2.0,beaver_D2,1859-01-21
3,1861-01-18,beaver,uts_beaver_S2,1,beaver_D2,1861-02-27,78.0,"(-110.7025292913094, 38.32662877359086)",BEAVER,uts_beaver,...,(<geoprocessing array object object at 0x00000...,uts_beaver_S2,1861-01-18,beaver,1862-01-16,1,2,2.0,beaver_D2,1859-01-21
4,1861-02-28,beaver,uts_beaver_S3,1,beaver_D2,1862-01-16,68.0,"(-112.03271613528274, 38.325452153781306)",BEAVER,uts_beaver,...,(<geoprocessing array object object at 0x00000...,uts_beaver_S3,1861-02-28,beaver,1862-01-16,1,2,2.0,beaver_D2,1859-01-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,1859-01-21,stmarys,utt_stmarys_S1,2,stmarys_D2,1861-01-17,61.0,"(-115.02048139314512, 40.691234378514096)",ST. MARY'S (ext),utt_stmarys,...,(<geoprocessing array object object at 0x00000...,utt_stmarys_S1,1856-01-05,stmarys,NaT,2,3,2.0,stmarys_D2,1859-01-21
3,1861-01-18,stmarys,utt_stmarys_S2,2,stmarys_D2,NaT,73.0,"(-115.49691574442528, 39.50992082631072)",ST. MARY'S (ext),utt_stmarys,...,(<geoprocessing array object object at 0x00000...,utt_stmarys_S2,1861-01-18,stmarys,NaT,2,3,2.0,stmarys_D2,1859-01-21
0,1856-01-12,shambip,utt_shambip_S1,n/a,n/a,1856-01-16,45.0,"(-113.12863371841395, 40.19848817754145)",SHAMBIP (ext),utt_shambip,...,(<geoprocessing array object object at 0x00000...,utt_shambip_S1,1856-01-12,NaN,NaT,NaN,NaN,NaN,NaN,NaT
1,1856-01-17,shambip,utt_shambip_S1,n/a,n/a,1861-01-17,45.0,"(-113.12863371841395, 40.19848817754145)",SHAMBIP (ext),utt_shambip,...,(<geoprocessing array object object at 0x00000...,utt_shambip_S1,1856-01-12,NaN,NaT,NaN,NaN,NaN,NaN,NaT


In [4]:
new_df = renamed_df.set_index(['CHANGE_DATE', 'DST_NUMBER'])

In [5]:
new_df.sort_index(inplace=True)
new_df

CHANGE_ID   county_version district_version  \
CHANGE_DATE DST_NUMBER                                               
1850-01-31  n/a             iron      uts_iron_S1              n/a   
            n/a         saltlake  uts_saltlake_S1              n/a   
            n/a          sanpete   uts_sanpete_S1              n/a   
            n/a           tooele    uts_tooele_S1              n/a   
            n/a             utah      uts_utah_S1              n/a   
...                          ...              ...              ...   
1988-04-25  8            daggett   uts_daggett_S2       daggett_D4   
            8           duchesne  uts_duchesne_S2      duchesne_D2   
            8             uintah    uts_uintah_S5        uintah_D3   
2003-01-01  7              emery     uts_emery_S5         emery_D2   
            7              grand     uts_grand_S3         grand_D2   

                       change_end_date    FID  \
CHANGE_DATE DST_NUMBER                          
1850-01-31  n/a             1850-12-02    9.0   
            n/a             1850-10-04   11.0   
            n/a             1851-08-07    7.0   
            n/a             1851-01-07    8.0   
            n/a             1851-01-07    6.0   
...                                ...    ...   
1988-04-25  8                      NaT  200.0   
            8                      NaT  194.0   
            8                      NaT  203.0   
2003-01-01  7                      NaT  211.0   
            7                      NaT  212.0   

                                                            Shape  \
CHANGE_DATE DST_NUMBER                                              
1850-01-31  n/a          (-112.81035978486156, 37.91571808636048)   
            n/a          (-111.90198101308543, 40.70626643625321)   
            n/a          (-111.54754342489007, 39.36017824691699)   
            n/a          (-112.41204100902685, 40.54703131531139)   
            n/a          (-111.69861046217375, 40.12623414962289)   
...                                                           ...   
1988-04-25  8            (-109.50624568182292, 40.88651480006792)   
            8           (-110.42475443917213, 40.297518598214516)   
            8            (-109.51862802066715, 40.12466957658222)   
2003-01-01  7           (-110.70061032158759, 38.996776615606805)   
            7              (-109.569650213519, 38.98174110833194)   

                                SHP_NAME        SHP_ID STATE SHP_FIPS  ...  \
CHANGE_DATE DST_NUMBER                                                 ...   
1850-01-31  n/a         LITTLE SALT LAKE      uts_iron    UT    49021  ...   
            n/a          GREAT SALT LAKE  uts_saltlake    UT    49035  ...   
            n/a                  SANPETE   uts_sanpete    UT    49039  ...   
            n/a                   TUILLA    uts_tooele    UT    49045  ...   
            n/a                     UTAH      uts_utah    UT    49049  ...   
...                                  ...           ...   ...      ...  ...   
1988-04-25  8                    DAGGETT   uts_daggett    UT    49009  ...   
            8                   DUCHESNE  uts_duchesne    UT    49013  ...   
            8                     UINTAH    uts_uintah    UT    49047  ...   
2003-01-01  7                      EMERY     uts_emery    UT    49015  ...   
            7                      GRAND     uts_grand    UT    49019  ...   

                                                                   SHAPE@  \
CHANGE_DATE DST_NUMBER                                                      
1850-01-31  n/a         (<geoprocessing array object object at 0x00000...   
            n/a         (<geoprocessing array object object at 0x00000...   
            n/a         (<geoprocessing array object object at 0x00000...   
            n/a         (<geoprocessing array object object at 0x00000...   
            n/a         (<geoprocessing array object object at 0x00000...   
...                   

In [6]:
idx = pd.IndexSlice
new_df.loc[idx[:, '1'], :]

CHANGE_ID     county_version district_version  \
CHANGE_DATE DST_NUMBER                                                   
1851-01-08  1             saltlake    uts_saltlake_S2      saltlake_D1   
            1               tooele      uts_tooele_S1        tooele_D1   
1851-10-04  1                davis       uts_davis_S1         davis_D2   
            1                 utah        uts_utah_S1          utah_D2   
            1                weber       uts_weber_S2         weber_D2   
1852-03-03  1                davis       uts_davis_S2         davis_D2   
            1             saltlake    uts_saltlake_S3      saltlake_D1   
            1               tooele      uts_tooele_S2        tooele_D1   
            1                 utah        uts_utah_S2          utah_D2   
            1                weber       uts_weber_S3         weber_D2   
1854-01-13  1                davis       uts_davis_S3         davis_D2   
            1             saltlake    uts_saltlake_S4      saltlake_D1   
            1                 utah        uts_utah_S3          utah_D2   
1854-01-17  1               tooele      uts_tooele_S3        tooele_D1   
1855-01-10  1                davis       uts_davis_S4         davis_D2   
            1             saltlake    uts_saltlake_S5      saltlake_D1   
            1                weber       uts_weber_S4         weber_D2   
1855-01-19  1               summit      uts_summit_S1        summit_D1   
            1               desert      utt_desert_S1        desert_D1   
            1           greenriver  utt_greenriver_S2    greenriver_D1   
1856-01-05  1               summit      uts_summit_S2        summit_D1   
            1               tooele      uts_tooele_S4        tooele_D1   
            1                 utah        uts_utah_S4          utah_D2   
            1                weber       uts_weber_S5         weber_D2   
            1               desert      utt_desert_S2        desert_D1   
1856-01-12  1               tooele      uts_tooele_S5        tooele_D1   
1856-01-17  1             boxelder    uts_boxelder_S1      boxelder_D1   
            1                cache       uts_cache_S1         cache_D1   
            1           greasewood  utt_greasewood_S1    greasewood_D1   
            1                malad       utt_malad_S1         malad_D1   
1859-01-21  1               beaver      uts_beaver_S1        beaver_D2   
            1                 iron        uts_iron_S5          iron_D3   
            1           washington  uts_washington_S2    washington_D2   
            1                cedar       utt_cedar_S1         cedar_D2   
1861-01-18  1               beaver      uts_beaver_S2        beaver_D2   
            1                 iron        uts_iron_S6          iron_D3   
            1           washington  uts_washington_S3    washington_D2   
1861-02-28  1               beaver      uts_beaver_S3        beaver_D2   
            1                 iron        uts_iron_S7          iron_D3   
            1           washington  uts_washington_S4    washington_D2   
1865-01-27  1                piute       uts_piute_S1         piute_D1   
1866-01-10  1                piute       uts_piute_S2         piute_D1   
1896-01-06  1             boxelder    uts_boxelder_S5      boxelder_D5   
            1                cache       uts_cache_S3         cache_D5   
            1                 rich        uts_rich_S4          rich_D4   
1917-05-08  1                 rich        uts_rich_S5          rich_D4   
1921-01-19  1                 rich        uts_rich_S6          rich_D4   

                       change_end_date    FID  \
CHANGE_DATE DST_NUMBER                          
1851-01-08  1               1852-03-02    1.0   
            1               1852-03-02    8.0   
1851-10-04  1               1852-03-02    3.0   
            1               1852-03-02    6.0   
            1               1852-03-02    4.0   
1852-03-03  1               1854-01-12   27.0   
      

In [7]:
new_df.groupby(level='CHANGE_DATE')['CHANGE_ID'].count().sort_values()

CHANGE_DATE
1880-01-20     1
1865-01-27     1
1874-02-12     1
1861-03-02     1
1892-03-09     1
              ..
1852-03-03    12
1862-01-17    17
1856-01-05    17
1859-01-21    20
1896-01-06    24
Name: CHANGE_ID, Length: 69, dtype: int64

In [8]:
new_df.groupby(level='DST_NUMBER')['CHANGE_ID'].count()


DST_NUMBER
1      47
1N      5
1S     46
2      64
3      61
4      19
5      10
6       9
7       9
8       3
n/a    36
Name: CHANGE_ID, dtype: int64

In [9]:
keys_df = renamed_df.set_index(['CHANGE_DATE', 'SHP_KEY', 'DST_KEY'])
keys_df

,,,CHANGE_ID,county_version,DST_NUMBER,district_version,change_end_date,FID,Shape,SHP_NAME,SHP_ID,STATE,...,CNTY_TYPE,FULL_NAME,SHAPE@,SHP_START_DATE,CountyName,DST_END_DATE,NewDistrict,OldDistrict,Version,DST_START_DATE
CHANGE_DATE,SHP_KEY,DST_KEY,,,,,,,,,,,,,,,,,,,,,
1856-01-05,uts_beaver_S1,NaN,beaver,uts_beaver_S1,n/a,n/a,1856-01-16,56.0,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,UT,...,County,BEAVER,(<geoprocessing array object object at 0x00000...,1856-01-05,NaN,NaT,NaN,NaN,NaN,NaT
1856-01-17,uts_beaver_S1,beaver_D1,beaver,uts_beaver_S1,2,beaver_D1,1859-01-20,56.0,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,UT,...,County,BEAVER,(<geoprocessing array object object at 0x00000...,1856-01-05,beaver,1859-01-20,2,2,1.0,1856-01-17
1859-01-21,uts_beaver_S1,beaver_D2,beaver,uts_beaver_S1,1,beaver_D2,1861-01-17,56.0,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,UT,...,County,BEAVER,(<geoprocessing array object object at 0x00000...,1856-01-05,beaver,1862-01-16,1,2,2.0,1859-01-21
1861-01-18,uts_beaver_S2,beaver_D2,beaver,uts_beaver_S2,1,beaver_D2,1861-02-27,78.0,"(-110.7025292913094, 38.32662877359086)",BEAVER,uts_beaver,UT,...,County,BEAVER,(<geoprocessing array object object at 0x00000...,1861-01-18,beaver,1862-01-16,1,2,2.0,1859-01-21
1861-02-28,uts_beaver_S3,beaver_D2,beaver,uts_beaver_S3,1,beaver_D2,1862-01-16,68.0,"(-112.03271613528274, 38.325452153781306)",BEAVER,uts_beaver,UT,...,County,BEAVER,(<geoprocessing array object object at 0x00000...,1861-02-28,beaver,1862-01-16,1,2,2.0,1859-01-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859-01-21,utt_stmarys_S1,stmarys_D2,stmarys,utt_stmarys_S1,2,stmarys_D2,1861-01-17,61.0,"(-115.02048139314512, 40.691234378514096)",ST. MARY'S (ext),utt_stmarys,UT,...,County,"ST. MARY'S (Utah Terr., extinct)",(<geoprocessing array object object at 0x00000...,1856-01-05,stmarys,NaT,2,3,2.0,1859-01-21
1861-01-18,utt_stmarys_S2,stmarys_D2,stmarys,utt_stmarys_S2,2,stmarys_D2,NaT,73.0,"(-115.49691574442528, 39.50992082631072)",ST. MARY'S (ext),utt_stmarys,UT,...,County,"ST. MARY'S (Utah Terr., extinct)",(<geoprocessing array object object at 0x00000...,1861-01-18,stmarys,NaT,2,3,2.0,1859-01-21
1856-01-12,utt_shambip_S1,NaN,shambip,utt_shambip_S1,n/a,n/a,1856-01-16,45.0,"(-113.12863371841395, 40.19848817754145)",SHAMBIP (ext),utt_shambip,UT,...,County,"SHAMBIP (Utah Terr., extinct)",(<geoprocessing array object object at 0x00000...,1856-01-12,NaN,NaT,NaN,NaN,NaN,NaT


In [10]:
keys_df.loc[idx[np.datetime64('1896-01-06'), :, :], :]

CHANGE_ID      county_version  \
CHANGE_DATE SHP_KEY            DST_KEY                                         
1896-01-06  uts_beaver_S7      beaver_D4          beaver       uts_beaver_S7   
            uts_boxelder_S5    boxelder_D5      boxelder     uts_boxelder_S5   
            uts_cache_S3       cache_D5            cache        uts_cache_S3   
            uts_carbon_S1      carbon_D2          carbon       uts_carbon_S1   
            uts_davis_S6       davis_D4            davis        uts_davis_S6   
            uts_emery_S4       emery_D2            emery        uts_emery_S4   
            uts_garfield_S2    garfield_D2      garfield     uts_garfield_S2   
            uts_grand_S2       grand_D2            grand        uts_grand_S2   
            uts_iron_S16       iron_D5              iron        uts_iron_S16   
            uts_juab_S9        juab_D3              juab         uts_juab_S9   
            uts_kane_S4        kane_D2              kane         uts_kane_S4   
            uts_millard_S9     millard_D3        millard      uts_millard_S9   
            uts_morgan_S1      morgan_D4          morgan       uts_morgan_S1   
            uts_piute_S5       piute_D3            piute        uts_piute_S5   
            uts_rich_S4        rich_D4              rich         uts_rich_S4   
            uts_sanjuan_S1     sanjuan_D3        sanjuan      uts_sanjuan_S1   
            uts_sanpete_S14    sanpete_D4        sanpete     uts_sanpete_S14   
            uts_sevier_S5      sevier_D2          sevier       uts_sevier_S5   
            uts_uintah_S2      uintah_D2          uintah       uts_uintah_S2   
            uts_utah_S7        utah_D5              utah         uts_utah_S7   
            uts_wasatch_S3     wasatch_D2        wasatch      uts_wasatch_S3   
            uts_washington_S14 washington_D4  washington  uts_washington_S14   
            uts_wayne_S1       wayne_D2            wayne        uts_wayne_S1   
            uts_weber_S7       weber_D6            weber        uts_weber_S7   

                                             DST_NUMBER district_version  \
CHANGE_DATE SHP_KEY            DST_KEY                                     
1896-01-06  uts_beaver_S7      beaver_D4              5        beaver_D4   
            uts_boxelder_S5    boxelder_D5            1      boxelder_D5   
            uts_cache_S3       cache_D5               1         cache_D5   
            uts_carbon_S1      carbon_D2              7        carbon_D2   
            uts_davis_S6       davis_D4               2         davis_D4   
            uts_emery_S4       emery_D2               7         emery_D2   
            uts_garfield_S2    garfield_D2            5      garfield_D2   
            uts_grand_S2       grand_D2               7         grand_D2   
            uts_iron_S16       iron_D5                5          iron_D5   
            uts_juab_S9        juab_D3                5          juab_D3   
            uts_kane_S4        kane_D2                6          kane_D2   
            uts_millard_S9     millard_D3             5       millard_D3   
            uts_morgan_S1      morgan_D4              2        morgan_D4   
            uts_piute_S5       piute_D3               6         piute_D3   
            uts_rich_S4        rich_D4                1          rich_D4   
            uts_sanjuan_S1     sanjuan_D3             7       sanjuan_D3   
            uts_sanpete_S14    sanpete_D4             7       sanpete_D4   
            uts_sevier_S5      sevier_D2              6        sevier_D2   
            uts_uintah_S2      uintah_D2              4        uintah_D2   
            uts_utah_S7        utah_D5                4          utah_D5   
            uts_wasatch_S3     wasatch_D2             4       wasatch_D2   
            uts_washington_S14 washington_D4          5    washington_D4   
            uts_wayne_S1       wayne_D2               6         wayne_D2   
            uts_weber_S7       weber_D6               2         we

In [17]:
renamed_df.iloc[-1, :].values[0] == np.datetime64('1861-01-18')

True

In [15]:
renamed_df.loc['0', :]

KeyError: '0'

# pd nulls

In [8]:
test_df = pd.DataFrame(
    data={
        'number': [None, None],
        'date': [None, None]
    }
)
test_df['number'] = test_df['number'].astype('float64')
test_df['date']= pd.to_datetime(test_df['date'])
test_df

,number,date
0,NaN,NaT
1,NaN,NaT


In [ ]:
name = 'uts_saltlake'


# District Dissolving

In [2]:
excel_file = r"C:\gis\Projects\HistoricCounties\Data\Exports\counties_and_districts_1.xlsx"
candy_df = pd.read_excel(excel_file)

In [3]:
candy_df.set_index(['CHANGE_DATE', 'DST_NUMBER'], drop=False, inplace=True)
candy_df.sort_index(inplace=True)
candy_df.iloc[[0, 10], :]

,,CHANGE_DATE,COUNTY_KEY,SHP_NAME,SHP_ID,SHP_FIPS,SHP_VERSION,DST_NAME,DST_NUMBER,SHP_START_DATE,SHP_END_DATE,DST_START_DATE,DST_END_DATE,SHP_CHANGE,SHP_CITATION,SHP_AREA_SQMI,SHP_KEY,DST_KEY
CHANGE_DATE,DST_NUMBER,,,,,,,,,,,,,,,,,
1850-01-30,NaN,1850-01-30,iron,LITTLE SALT LAKE,uts_iron,49021,1.0,NaN,NaN,1850-01-30,1850-12-01,NaT,NaT,LITTLE SALT LAKE (now IRON) created by the sta...,"(Morgan, 47-48, 145)",756.0,uts_iron_S1,NaN
1851-01-07,1,1851-01-07,saltlake,GREAT SALT LAKE,uts_saltlake,49035,2.0,Salt Lake,1,1850-10-04,1852-03-01,1851-01-07,1856-01-15,GREAT SALT LAKE (now SALT LAKE) lost to creati...,"(Morgan, 190)",732.0,uts_saltlake_S2,salt lake_D1


In [4]:
candy_df.loc[("1850-01-30")]

CHANGE_DATE COUNTY_KEY          SHP_NAME        SHP_ID  \
CHANGE_DATE DST_NUMBER                                                          
1850-01-30  NaN         1850-01-30       iron  LITTLE SALT LAKE      uts_iron   
            NaN         1850-01-30   saltlake   GREAT SALT LAKE  uts_saltlake   
            NaN         1850-01-30    sanpete           SANPETE   uts_sanpete   
            NaN         1850-01-30     tooele            TUILLA    uts_tooele   
            NaN         1850-01-30       utah              UTAH      uts_utah   
            NaN         1850-01-30      weber             WEBER     uts_weber   

                       SHP_FIPS  SHP_VERSION DST_NAME DST_NUMBER  \
CHANGE_DATE DST_NUMBER                                             
1850-01-30  NaN           49021          1.0      NaN        NaN   
            NaN           49035          1.0      NaN        NaN   
            NaN           49039          1.0      NaN        NaN   
            NaN           49045          1.0      NaN        NaN   
            NaN           49049          1.0      NaN        NaN   
            NaN           49057          1.0      NaN        NaN   

                       SHP_START_DATE SHP_END_DATE DST_START_DATE  \
CHANGE_DATE DST_NUMBER                                              
1850-01-30  NaN            1850-01-30   1850-12-01            NaT   
            NaN            1850-01-30   1850-10-03            NaT   
            NaN            1850-01-30   1852-03-01            NaT   
            NaN            1850-01-30   1852-03-01            NaT   
            NaN            1850-01-30   1852-03-01            NaT   
            NaN            1850-01-30   1850-10-03            NaT   

                       DST_END_DATE  \
CHANGE_DATE DST_NUMBER                
1850-01-30  NaN                 NaT   
            NaN                 NaT   
            NaN                 NaT   
            NaN                 NaT   
            NaN                 NaT   
            NaN                 NaT   

                                                               SHP_CHANGE  \
CHANGE_DATE DST_NUMBER                                                      
1850-01-30  NaN         LITTLE SALT LAKE (now IRON) created by the sta...   
            NaN         GREAT SALT LAKE (now SALT LAKE) created by the...   
            NaN         SANPETE created by the state of Deseret to cov...   
            NaN         TOOELE (spelled TUILLA here) created by the st...   
            NaN         UTAH created by the state of Deseret to cover ...   
            NaN                    WEBER created by the state of Deseret.   

                                                    SHP_CITATION  \
CHANGE_DATE DST_NUMBER                                             
1850-01-30  NaN                             (Morgan, 47-48, 145)   
            NaN         (Morgan, 47-48, 144; HRS Utah, Weber, 9)   
            NaN                             (Morgan, 47-48, 144)   
            NaN                             (Morgan, 47-48, 145)   
            NaN                             (Morgan, 47-48, 144)   
            NaN         (Morgan, 47-48, 144; HRS Utah, Weber, 9)   

                        SHP_AREA_SQMI          SHP_KEY DST_KEY  
CHANGE_DATE DST_NUMBER                                          
1850-01-30  NaN                 756.0      uts_iron_S1     NaN  
            NaN                 895.0  uts_saltlake_S1     NaN  
            NaN                1401.0   uts_sanpete_S1     NaN  
            NaN                 591.0    uts_tooele_S1     NaN  
            NaN                2081.0      uts_utah_S1     NaN  
            NaN                 388.0     uts_weber_S1     NaN

In [5]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
candy_df

CHANGE_DATE  COUNTY_KEY              SHP_NAME  \
CHANGE_DATE DST_NUMBER                                                 
1850-01-30  NaN         1850-01-30        iron      LITTLE SALT LAKE   
            NaN         1850-01-30    saltlake       GREAT SALT LAKE   
            NaN         1850-01-30     sanpete               SANPETE   
            NaN         1850-01-30      tooele                TUILLA   
            NaN         1850-01-30        utah                  UTAH   
            NaN         1850-01-30       weber                 WEBER   
1850-10-04  NaN         1850-10-04       davis                 DAVIS   
            NaN         1850-10-04    saltlake       GREAT SALT LAKE   
            NaN         1850-10-04       weber                 WEBER   
1850-12-02  NaN         1850-12-02        iron                  IRON   
1851-01-07  1           1851-01-07    saltlake       GREAT SALT LAKE   
            1           1851-01-07      tooele                TUILLA   
            3           1851-01-07        utah                  UTAH   
1851-08-07  2           1851-08-07       davis                 DAVIS   
            2           1851-08-07       weber                 WEBER   
            3           1851-08-07        iron                  IRON   
            3           1851-08-07     sanpete               SANPETE   
1851-10-03  1           1851-10-03       davis                 DAVIS   
            1           1851-10-03        utah                  UTAH   
            1           1851-10-03       weber                 WEBER   
            2           1851-10-03     millard               MILLARD   
            2           1851-10-03     sanpete               SANPETE   
1852-03-02  1           1852-03-02       davis                 DAVIS   
            1           1852-03-02    saltlake       GREAT SALT LAKE   
            1           1852-03-02      tooele                TOOELE   
            1           1852-03-02        utah                  UTAH   
            1           1852-03-02       weber                 WEBER   
            2           1852-03-02     millard               MILLARD   
            2           1852-03-02     sanpete               SANPETE   
            3           1852-03-02        iron                  IRON   
            NaN         1852-03-02        juab                  JUAB   
            NaN         1852-03-02  washington            WASHINGTON   
            NaN         1852-03-02      desert  DESERT (Ut Ter, ext)   
            NaN         1852-03-02  greenriver     GREEN RIVER (ext)   
1854-01-12  1           1854-01-12       davis                 DAVIS   
            1           1854-01-12    saltlake       GREAT SALT LAKE   
            1           1854-01-12        utah                  UTAH   
            2           1854-01-12        juab                  JUAB   
            2           1854-01-12     sanpete               SANPETE   
            NaN         1854-01-12      summit                SUMMIT   
            NaN         1854-01-12  greenriver     GREEN RIVER (ext)   
1854-01-16  1           1854-01-16      tooele                TOOELE   
            2           1854-01-16        juab                  JUAB   
            2           1854-01-16     millard               MILLARD   
            3           1854-01-16        iron                  IRON   
            NaN         1854-01-16      carson  CARSON (Ut Ter, ext)   
1855-01-09  1           1855-01-09       davis                 DAVIS   
            1           1855-01-09    saltlake       GREAT SALT LAKE   
            1           1855-01-09       weber                 WEBER   
1855-01-17  2           1855-01-17        juab                  JUAB   
            2           1855-01-17     sanpete               SANPETE   
1855-01-18  1           1855-01-18      summit                SUMMIT   
            1           1855-01-18      desert  DESERT (Ut Ter, ext)   
            1           1855-01-18  greenriver     GREEN RIVER 

In [6]:
duchesne_df = candy_df[candy_df['DST_NAME'] == 'Duchesne'].copy()
duchesne_df['combined_key'] = duchesne_df['SHP_KEY'] + '_' + duchesne_df['DST_KEY']
duchesne_df

,,CHANGE_DATE,COUNTY_KEY,SHP_NAME,SHP_ID,SHP_FIPS,SHP_VERSION,DST_NAME,DST_NUMBER,SHP_START_DATE,SHP_END_DATE,DST_START_DATE,DST_END_DATE,SHP_CHANGE,SHP_CITATION,SHP_AREA_SQMI,SHP_KEY,DST_KEY,combined_key
CHANGE_DATE,DST_NUMBER,,,,,,,,,,,,,,,,,,
1914-08-13,4,1914-08-13,duchesne,NaN,NaN,NaN,NaN,Duchesne,4,NaT,NaT,1914-08-13,1988-04-24,NaN,NaN,NaN,NaN,duchesne_D1,NaN
1915-01-04,4,1915-01-04,duchesne,DUCHESNE,uts_duchesne,49013,1.0,Duchesne,4,1915-01-04,1917-03-04,1914-08-13,1988-04-24,DUCHESNE created from WASATCH. Referendum held...,"(Governor's Proclamation, 13 August 1914; Utah...",3163.0,uts_duchesne_S1,duchesne_D1,uts_duchesne_S1_duchesne_D1
1917-03-05,4,1917-03-05,duchesne,DUCHESNE,uts_duchesne,49013,2.0,Duchesne,4,1917-03-05,2003-12-31,1914-08-13,1988-04-24,DUCHESNE gained from UINTAH.,"(Utah Laws 1917, 12th sess., ch. 34, pp. 102-104)",3256.0,uts_duchesne_S2,duchesne_D1,uts_duchesne_S2_duchesne_D1
1988-04-25,8,1988-04-25,duchesne,DUCHESNE,uts_duchesne,49013,2.0,Duchesne,8,1917-03-05,2003-12-31,1988-04-25,NaT,DUCHESNE gained from UINTAH.,"(Utah Laws 1917, 12th sess., ch. 34, pp. 102-104)",3256.0,uts_duchesne_S2,duchesne_D2,uts_duchesne_S2_duchesne_D2


In [7]:
candy_df.groupby('COUNTY_KEY')['COUNTY_KEY'].count().max()

21

In [99]:
def _lookup_rows_present_in_date(data_df, date):
    key_list = []
    for row in list(data_df.values):  #: [(start, end, key), ...]
        if row[0] <= np.datetime64(date) < row[1]:
            key_list.append(row[2])
            continue
        if row[0] <= np.datetime64(date) and pd.isnull(row[1]):
            key_list.append(row[2])
            continue
    return key_list

In [2]:
excel_file = r"C:\gis\Projects\HistoricCounties\Data\Exports\counties_and_districts_2.xlsx"
all_df = pd.read_excel(excel_file)
four_df = all_df[all_df['DST_NUMBER'] == '4'].copy()
four_df['UNIQUE_ROW_KEY'] = [str(shp_key) + '__' + str(dst_key) 
                              for shp_key, dst_key 
                              in zip(four_df['SHP_KEY'], four_df['DST_KEY'])]
# four_df['CHANGE_END_DATE'] = four_df['CHANGE_DATE'].shift(-1) - pd.Timedelta(days=1)

In [108]:
dates = pd.DataFrame(four_df['CHANGE_DATE'].unique(), columns=['CHANGE_DATE'])
dates['CHANGE_END_DATE'] = dates['CHANGE_DATE'].shift(-1) - pd.Timedelta(days=1)

cd_dict = {date: _lookup_rows_present_in_date(four_df[['CHANGE_DATE', 'CHANGE_END_DATE', 'UNIQUE_ROW_KEY']], date) 
           for date in dates['CHANGE_DATE']}
# cd_df = pd.DataFrame.from_dict(cd_dict)
cd_df = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in cd_dict.items()]))
cd_df.T.sort_index()

,0,1,2,3,4,5,6
1892-03-09,uts_boxelder_S5__box elder_D4,uts_cache_S3__cache_D4,uts_morgan_S1__morgan_D3,uts_rich_S4__rich_D3,uts_weber_S7__weber_D5,NaN,NaN
1896-01-05,uts_uintah_S2__uintah_D2,uts_utah_S7__utah_D5,uts_wasatch_S3__wasatch_D2,NaN,NaN,NaN,NaN
1914-08-13,nan__duchesne_D1,uts_uintah_S2__uintah_D2,uts_utah_S7__utah_D5,uts_wasatch_S3__wasatch_D2,NaN,NaN,NaN
1915-01-04,uts_duchesne_S1__duchesne_D1,uts_uintah_S2__uintah_D2,uts_utah_S7__utah_D5,uts_wasatch_S4__wasatch_D2,NaN,NaN,NaN
1917-03-05,uts_duchesne_S2__duchesne_D1,uts_uintah_S3__uintah_D2,uts_utah_S7__utah_D5,uts_wasatch_S4__wasatch_D2,NaN,NaN,NaN
1918-01-07,uts_duchesne_S2__duchesne_D1,uts_uintah_S4__uintah_D2,uts_utah_S7__utah_D5,uts_wasatch_S4__wasatch_D2,NaN,NaN,NaN
1919-12-31,uts_duchesne_S2__duchesne_D1,uts_uintah_S5__uintah_D2,uts_utah_S7__utah_D5,uts_wasatch_S4__wasatch_D2,NaN,NaN,NaN
1969-03-14,uts_daggett_S2__daggett_D3,uts_duchesne_S2__duchesne_D1,uts_summit_S10__summit_D3,uts_uintah_S5__uintah_D2,uts_utah_S7__utah_D5,uts_wasatch_S4__wasatch_D2,NaN
1976-02-10,uts_daggett_S2__daggett_D3,uts_duchesne_S2__duchesne_D1,uts_juab_S11__juab_D4,uts_uintah_S5__uintah_D2,uts_utah_S7__utah_D5,uts_wasatch_S4__wasatch_D2,NaN
1982-02-16,uts_daggett_S2__daggett_D3,uts_duchesne_S2__duchesne_D1,uts_juab_S11__juab_D4,uts_millard_S11__millard_D4,uts_uintah_S5__uintah_D2,uts_utah_S7__utah_D5,uts_wasatch_S4__wasatch_D2


In [3]:
four_df

,OBJECTID,Shape_Length,Shape_Area,CHANGE_DATE,CHANGE_END_DATE,COUNTY_KEY,SHP_NAME,SHP_ID,SHP_FIPS,SHP_VERSION,...,SHP_START_DATE,SHP_END_DATE,DST_START_DATE,DST_END_DATE,SHP_CHANGE,SHP_CITATION,SHP_AREA_SQMI,SHP_KEY,DST_KEY,UNIQUE_ROW_KEY
18,19,6.194057,1.880056,1892-03-09,1896-01-04,boxelder,BOX ELDER,uts_boxelder,49003,5.0,...,1880-02-17,2003-12-31,1892-03-09,1896-01-04,BOX ELDER gained part of the islands and water...,"(Utah Terr. Laws 1880, 24th sess., ch. 14, sec...",6725.0,uts_boxelder_S5,box elder_D4,uts_boxelder_S5__box elder_D4
26,27,3.059603,0.328738,1892-03-09,1896-01-04,cache,CACHE,uts_cache,49005,3.0,...,1864-01-15,2003-12-31,1892-03-09,1896-01-04,CACHE lost to creation of RICHLAND (now RICH);...,"(Utah Terr. Laws 1863, 13th sess., pp. 18-19)",1173.0,uts_cache_S3,cache_D4,uts_cache_S3__cache_D4
34,35,2.914011,0.200086,1969-03-14,1988-04-24,daggett,DAGGETT,uts_daggett,49009,2.0,...,1919-12-31,2003-12-31,1969-03-14,1988-04-24,"In 1919, DAGGETT lost to UINTAH.","(HRS Utah, Daggett, 12, 15; HRS Utah, Uintah, ...",723.0,uts_daggett_S2,daggett_D3,uts_daggett_S2__daggett_D3
46,47,0.000000,0.000000,1914-08-13,1915-01-03,duchesne,NaN,NaN,NaN,NaN,...,NaT,NaT,1914-08-13,1988-04-24,NaN,NaN,NaN,NaN,duchesne_D1,nan__duchesne_D1
47,48,3.916392,0.867740,1915-01-04,1917-03-04,duchesne,DUCHESNE,uts_duchesne,49013,1.0,...,1915-01-04,1917-03-04,1914-08-13,1988-04-24,DUCHESNE created from WASATCH. Referendum held...,"(Governor's Proclamation, 13 August 1914; Utah...",3163.0,uts_duchesne_S1,duchesne_D1,uts_duchesne_S1__duchesne_D1
48,49,3.973802,0.893222,1917-03-05,1988-04-24,duchesne,DUCHESNE,uts_duchesne,49013,2.0,...,1917-03-05,2003-12-31,1914-08-13,1988-04-24,DUCHESNE gained from UINTAH.,"(Utah Laws 1917, 12th sess., ch. 34, pp. 102-104)",3256.0,uts_duchesne_S2,duchesne_D1,uts_duchesne_S2__duchesne_D1
99,100,6.900285,0.926496,1976-02-10,NaT,juab,JUAB,uts_juab,49023,11.0,...,1919-03-08,2003-12-31,1976-02-10,NaT,"JUAB exchanged with SANPETE, lost to MILLARD w...","(Utah Laws 1919, 13th sess., ch. 27, pp. 47-49)",3406.0,uts_juab_S11,juab_D4,uts_juab_S11__juab_D4
119,120,6.198726,1.840776,1982-02-16,NaT,millard,MILLARD,uts_millard,49027,11.0,...,1921-03-10,2003-12-31,1982-02-16,NaT,MILLARD exchanged with SEVIER.,"(Utah Laws 1921, 14th sess., ch. 33, pp. 104-106)",6828.0,uts_millard_S11,millard_D4,uts_millard_S11__millard_D4
122,123,2.855789,0.169535,1892-03-09,1896-01-04,morgan,MORGAN,uts_morgan,49029,1.0,...,1862-01-16,2003-12-31,1892-03-09,1896-01-04,"MORGAN created by Utah Territory from DAVIS, G...","(Utah Terr. Laws 1861, 11th sess., sec. 15/p. 50)",611.0,uts_morgan_S1,morgan_D3,uts_morgan_S1__morgan_D3
139,140,3.049769,0.304004,1892-03-09,1896-01-04,rich,RICH,uts_rich,49033,4.0,...,1872-02-15,1917-05-07,1892-03-09,1896-01-04,RICH gained small area from SUMMIT near town o...,"(Utah Terr. Laws 1872, 20th sess., ch. 25, p. 40)",1086.0,uts_rich_S4,rich_D3,uts_rich_S4__rich_D3


In [9]:
test_list = ['version' for i in range(four_df.shape[0])]
test_list
four_df['versions2'] = ['foo' for i in range(four_df.shape[0])]
four_df

CHANGE_DATE COUNTY_KEY   SHP_NAME        SHP_ID  \
CHANGE_DATE DST_NUMBER                                                   
1892-03-09  4           1892-03-09   boxelder  BOX ELDER  uts_boxelder   
            4           1892-03-09      cache      CACHE     uts_cache   
            4           1892-03-09     morgan     MORGAN    uts_morgan   
            4           1892-03-09       rich       RICH      uts_rich   
            4           1892-03-09      weber      WEBER     uts_weber   
1896-01-05  4           1896-01-05     uintah     UINTAH    uts_uintah   
            4           1896-01-05       utah       UTAH      uts_utah   
            4           1896-01-05    wasatch    WASATCH   uts_wasatch   
1914-08-13  4           1914-08-13   duchesne        NaN           NaN   
1915-01-04  4           1915-01-04   duchesne   DUCHESNE  uts_duchesne   
            4           1915-01-04    wasatch    WASATCH   uts_wasatch   
1917-03-05  4           1917-03-05   duchesne   DUCHESNE  uts_duchesne   
            4           1917-03-05     uintah     UINTAH    uts_uintah   
1918-01-07  4           1918-01-07     uintah     UINTAH    uts_uintah   
1919-12-31  4           1919-12-31     uintah     UINTAH    uts_uintah   
1969-03-14  4           1969-03-14    daggett    DAGGETT   uts_daggett   
            4           1969-03-14     summit     SUMMIT    uts_summit   
1976-02-10  4           1976-02-10       juab       JUAB      uts_juab   
1982-02-16  4           1982-02-16    millard    MILLARD   uts_millard   

                       SHP_FIPS  SHP_VERSION   DST_NAME DST_NUMBER  \
CHANGE_DATE DST_NUMBER                                               
1892-03-09  4             49003          5.0  Box Elder          4   
            4             49005          3.0      Cache          4   
            4             49029          1.0     Morgan          4   
            4             49033          4.0       Rich          4   
            4             49057          7.0      Weber          4   
1896-01-05  4             49047          2.0     Uintah          4   
            4             49049          7.0       Utah          4   
            4             49051          3.0    Wasatch          4   
1914-08-13  4               NaN          NaN   Duchesne          4   
1915-01-04  4             49013          1.0   Duchesne          4   
            4             49051          4.0    Wasatch          4   
1917-03-05  4             49013          2.0   Duchesne          4   
            4             49047          3.0     Uintah          4   
1918-01-07  4             49047          4.0     Uintah          4   
1919-12-31  4             49047          5.0     Uintah          4   
1969-03-14  4             49009          2.0    Daggett          4   
            4             49043         10.0     Summit          4   
1976-02-10  4             49023         11.0       Juab          4   
1982-02-16  4             49027         11.0    Millard          4   

                       SHP_START_DATE SHP_END_DATE DST_START_DATE  \
CHANGE_DATE DST_NUMBER                                              
1892-03-09  4              1880-02-17   2003-12-31     1892-03-09   
            4              1864-01-15   2003-12-31     1892-03-09   
            4              1862-01-16   2003-12-31     1892-03-09   
            4              1872-02-15   1917-05-07     1892-03-09   
            4              1880-02-17   2003-12-31     1892-03-09   
1896-01-05  4              1892-02-26   1917-03-04     1896-01-05   
            4              1884-03-12   2003-12-31     1896-01-05   
            4              1884-03-12   1915-01-03     1896-01-05   
1914-08-13  4                     NaT          NaT     1914-08-13   
1915-01-04  4              1915-01-04   1917-03-04     1914-08-13   
            4              1915-01-04   2003-12-31     1896-01-05   
1917-03-05  4              1917-03-05   2003-12-31     1914-08-13   
            4              1917

In [26]:
district_versions_df = pd.read_pickle(r'C:\gis\Projects\HistoricCounties\Data\JudicialDistricts\district_versions.pkl')
district_versions_df.to_csv(r'C:\gis\Projects\HistoricCounties\Data\JudicialDistricts\district_versions.csv')
district_versions_df

,date,county_name,county_version,district_number,district_version,change_end_date,FID,Shape,NAME,ID,STATE,FIPS,VERSION,END_DATE,CHANGE,CITATION,START_N,END_N,AREA_SQMI,DATASET,CNTY_TYPE,FULL_NAME,SHAPE@,shape_key,county_key_x,START_DATE,CountyName,EndDate,NewDistrict,OldDistrict,Version,county_key_y,district_key,StartDate,district_version_key
0,1856-01-05,beaver,uts_beaver_S1,n/a,1,1856-01-16,56.0,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,UT,49001,1.0,1861-01-17,BEAVER created by Utah Territory from IRON; BE...,"(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)",18560105.0,18610117.0,15141.0,UT_Historical_Counties,County,BEAVER,(<geoprocessing array object object at 0x00000...,uts_beaver_S1,beaver,1856-01-05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,Dn/a_V01
1,1856-01-05,boxelder,uts_boxelder_S1,n/a,1,1856-01-16,52.0,"(-112.2645534049489, 41.55680039383712)",BOX ELDER,uts_boxelder,UT,49003,1.0,1862-01-16,BOX ELDER created by Utah Territory from WEBER.,"(Utah Terr. Laws 1855, 5th sess., sec. 6/pp. 6-7)",18560105.0,18620116.0,1634.0,UT_Historical_Counties,County,BOX ELDER,(<geoprocessing array object object at 0x00000...,uts_boxelder_S1,boxelder,1856-01-05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,Dn/a_V01
2,1856-01-05,cache,uts_cache_S1,n/a,1,1856-01-16,49.0,"(-111.46127346514218, 41.802863887382756)",CACHE,uts_cache,UT,49005,1.0,1862-01-16,CACHE created by Utah Territory from SUMMIT an...,"(Utah Terr. Laws 1855, 5th sess., sec. 5/p. 6)",18560105.0,18620116.0,1706.0,UT_Historical_Counties,County,CACHE,(<geoprocessing array object object at 0x00000...,uts_cache_S1,cache,1856-01-05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,Dn/a_V01
3,1850-10-05,davis,uts_davis_S1,n/a,1,1851-08-07,3.0,"(-111.93796626587579, 41.00857364764544)",DAVIS,uts_davis,UT,49011,1.0,1852-03-02,DAVIS created by the state of Deseret from GRE...,"(Morgan, 54-55, 152-153)",18501005.0,18520302.0,257.0,UT_Historical_Counties,County,DAVIS,(<geoprocessing array object object at 0x00000...,uts_davis_S1,davis,1850-10-05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,Dn/a_V01
4,1882-03-09,garfield,uts_garfield_S1,n/a,1,1884-03-12,164.0,"(-111.41806291922869, 37.858706584285294)",GARFIELD,uts_garfield,UT,49017,1.0,1884-03-12,GARFIELD created by Utah Territory from IRON.,"(Utah Terr. Laws 1882, 25th sess., ch. 52, pp....",18820309.0,18840312.0,5100.0,UT_Historical_Counties,County,GARFIELD,(<geoprocessing array object object at 0x00000...,uts_garfield_S1,garfield,1882-03-09,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,Dn/a_V01
5,1884-03-13,garfield,uts_garfield_S2,n/a,2,1892-03-08,168.0,"(-111.44312781220562, 37.854917540244585)",GARFIELD,uts_garfield,UT,49017,2.0,2003-12-31,GARFIELD gained from IRON.,"(Utah Terr. Laws 1884, 26th sess., ch. 40, p. 58)",18840313.0,20031231.0,5208.0,UT_Historical_Counties,County,GARFIELD,(<geoprocessing array object object at 0x00000...,uts_garfield_S2,garfield,1884-03-13,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,Dn/a_V02
6,1850-01-31,iron,uts_iron_S1,n/a,1,1850-12-02,9.0,"(-112.81035978486156, 37.91571808636048)",LITTLE SALT LAKE,uts_iron,UT,49021,1.0,1850-12-02,LITTLE SALT LAKE (now IRON) created by the sta...,"(Morgan, 47-48, 145)",18500131.0,18501202.0,756.0,UT_Historical_Counties,County,LITTLE SALT LAKE,(<geoprocessing array object object at 0x00000...,uts_iron_S1,iron,1850-01-31,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,Dn/a_V01
7,1850-12-03,iron,uts_iron_S2,n/a,2,1851-08-07,2.0,"(-112.4800682659367, 37.830956304044605)",IRON,uts_iron,UT,49021,2.0,1852-03-02,LITTLE SALT LAKE renamed IRON and gained non-c...,"(Morgan, 56-57, 157)",18501203.0,18520302.0,2756.0,UT_Historical_Counties,County,IRON,(<geoprocessing array object object at 0x00000...,uts_iron_S2,iron,1850-12-03,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,Dn/a_V02
8,1852-03-03,juab,uts_juab_S1,n/a,1,1854-01-12,19.0,"(-115.85228044720164, 39.67470375720514)",JUAB,uts_juab,UT,49023,1.0,1854-01-16,JUAB created by Utah Territory from UTAH and n...,"(Utah Terr. Laws 1851, 1st sess., sec. 9/p. 163)",18520303.0,18540116.0,17447.0,UT_Historical_Counties,Coun

In [27]:
sorted_district_versions_df = district_versions_df.set_index(['district_version_key', 'county_name'], drop=False)
sorted_district_versions_df.sort_index(inplace=True)
sorted_district_versions_df

date county_name      county_version  \
district_version_key county_name                                              
D1N_V01              boxelder    1880-02-20    boxelder     uts_boxelder_S5   
                     cache       1880-02-20       cache        uts_cache_S3   
                     morgan      1880-01-20      morgan       uts_morgan_S1   
                     rich        1880-02-20        rich         uts_rich_S4   
                     weber       1880-02-20       weber        uts_weber_S7   
D1S_V01              carbon      1894-03-08      carbon       uts_carbon_S1   
                     emery       1880-02-12       emery        uts_emery_S1   
                     grand       1890-03-13       grand        uts_grand_S1   
                     juab        1859-01-21        juab         uts_juab_S4   
                     millard     1859-01-21     millard      uts_millard_S3   
                     sanjuan     1892-03-10     sanjuan      uts_sanjuan_S1   
                     sanpete     1859-01-21     sanpete      uts_sanpete_S4   
                     sevier      1865-12-27      sevier       uts_sevier_S1   
                     uintah      1880-02-18      uintah       uts_uintah_S1   
                     utah        1859-01-21        utah         uts_utah_S4   
                     wasatch     1862-01-17     wasatch      uts_wasatch_S1   
D1S_V02              emery       1880-02-17       emery        uts_emery_S2   
                     grand       1892-02-27       grand        uts_grand_S2   
                     juab        1861-01-18        juab         uts_juab_S5   
                     millard     1861-01-18     millard      uts_millard_S4   
                     sanpete     1861-02-28     sanpete      uts_sanpete_S5   
                     sevier      1866-01-10      sevier       uts_sevier_S2   
                     uintah      1892-02-27      uintah       uts_uintah_S2   
                     utah        1862-01-17        utah         uts_utah_S5   
                     wasatch     1880-02-18     wasatch      uts_wasatch_S2   
D1S_V03              emery       1890-03-13       emery        uts_emery_S3   
                     juab        1862-01-17        juab         uts_juab_S6   
                     millard     1862-01-17     millard      uts_millard_S5   
                     sanpete     1862-01-17     sanpete      uts_sanpete_S6   
                     sevier      1880-02-12      sevier       uts_sevier_S3   
                     utah        1880-02-18        utah         uts_utah_S6   
                     wasatch     1884-03-13     wasatch      uts_wasatch_S3   
D1S_V04              emery       1894-03-08       emery        uts_emery_S4   
                     juab        1866-01-10        juab         uts_juab_S7   
                     millard     1862-07-14     millard      uts_millard_S6   
                     sanpete     1865-01-16     sanpete      uts_sanpete_S7   
                     sevier      1880-02-17      sevier       uts_sevier_S4   
                     utah        1884-03-13        utah         uts_utah_S7   
D1S_V05              juab        1870-02-18        juab         uts_juab_S8   
                     millard     1866-01-10     millard      uts_millard_S7   
                     sanpete     1866-01-10     sanpete      uts_sanpete_S8   
                     sevier      1890-03-10      sevier       uts_sevier_S5   
D1S_V06              juab        1888-03-06        juab         uts_juab_S9   
                     millard     1866-05-05     millard      uts_millard_S8   
                     sanpete     1880-02-12     sanpete      uts_sanpete_S9   
D1S_V07              millard     1888-03-06     millard      uts_millard_S9   
                     sanpete     1880-02-17     sanpete     uts_sanpete_S10   
D1S_V08              sanpete     1880-02-18     sanpete     uts_sanpete_S11   
D1S_V09              sanpete     1888-03-06     sanpete     uts_sanpete_S12   
D1S_V10   

# Random dict/list stuff

In [15]:
test_list = [1, 2, 3, 3, 4]
def test_function(uniques, number):
    if number not in uniques:
        uniques.append(number)
        return 'Unique'
    return 'Duplicate detected'

uniques = []
run = [test_function(uniques, n) for n in test_list]
run

['Unique', 'Unique', 'Unique', 'Duplicate detected', 'Unique']

In [18]:
# test_dict = {1: []}
test_names = ['foo', 'bar', 'baz', 'foo', 'bar', 'foo']
for name in test_names:
    current_number = 1
    for number in test_dict:
        if name not in versions[number]:
            versions[number].append(name)
            break

test_dict

{1: ['foo', 'bar', 'baz']}

In [20]:
numdict = {1: 'fo', 2: "bar", 5: "baz", 3: 'moo'}
max(numdict.keys())

5

# District Dissolve 2

In [2]:
excel_file = r"C:\gis\Projects\HistoricCounties\Data\Exports\counties_and_districts_2.xlsx"
all_df = pd.read_excel(excel_file)
four_df = all_df[all_df['DST_NUMBER'] == '4'].copy()
four_df['UNIQUE_ROW_KEY'] = [str(shp_key) + '__' + str(dst_key) 
                              for shp_key, dst_key 
                              in zip(four_df['SHP_KEY'], four_df['DST_KEY'])]

In [3]:
dates = four_df.set_index(['CHANGE_DATE', 'CHANGE_END_DATE'], drop=False)
dates.sort_index(inplace=True)
dates

OBJECTID  Shape_Length  Shape_Area CHANGE_DATE  \
CHANGE_DATE CHANGE_END_DATE                                                   
1892-03-09  1896-01-04             19      6.194057    1.880056  1892-03-09   
            1896-01-04             27      3.059603    0.328738  1892-03-09   
            1896-01-04            123      2.855789    0.169535  1892-03-09   
            1896-01-04            140      3.049769    0.304004  1892-03-09   
            1896-01-04            268      2.978142    0.183538  1892-03-09   
1896-01-05  1915-01-03            236      5.672699    1.199617  1896-01-05   
            1917-03-04            216      5.501336    1.456733  1896-01-05   
            NaT                   232      5.069886    0.585836  1896-01-05   
1914-08-13  1915-01-03             47      0.000000    0.000000  1914-08-13   
1915-01-04  1917-03-04             48      3.916392    0.867740  1915-01-04   
            NaT                   237      3.335081    0.331877  1915-01-04   
1917-03-05  1918-01-06            217      5.446051    1.426954  1917-03-05   
            1988-04-24             49      3.973802    0.893222  1917-03-05   
1918-01-07  1919-12-30            218      5.159772    1.196196  1918-01-07   
1919-12-31  1988-04-24            219      5.387340    1.231166  1919-12-31   
1969-03-14  1975-03-20            199      5.091970    0.520679  1969-03-14   
            1988-04-24             35      2.914011    0.200086  1969-03-14   
1976-02-10  NaT                   100      6.900285    0.926496  1976-02-10   
1982-02-16  NaT                   120      6.198726    1.840776  1982-02-16   

                            CHANGE_END_DATE COUNTY_KEY   SHP_NAME  \
CHANGE_DATE CHANGE_END_DATE                                         
1892-03-09  1896-01-04           1896-01-04   boxelder  BOX ELDER   
            1896-01-04           1896-01-04      cache      CACHE   
            1896-01-04           1896-01-04     morgan     MORGAN   
            1896-01-04           1896-01-04       rich       RICH   
            1896-01-04           1896-01-04      weber      WEBER   
1896-01-05  1915-01-03           1915-01-03    wasatch    WASATCH   
            1917-03-04           1917-03-04     uintah     UINTAH   
            NaT                         NaT       utah       UTAH   
1914-08-13  1915-01-03           1915-01-03   duchesne        NaN   
1915-01-04  1917-03-04           1917-03-04   duchesne   DUCHESNE   
            NaT                         NaT    wasatch    WASATCH   
1917-03-05  1918-01-06           1918-01-06     uintah     UINTAH   
            1988-04-24           1988-04-24   duchesne   DUCHESNE   
1918-01-07  1919-12-30           1919-12-30     uintah     UINTAH   
1919-12-31  1988-04-24           1988-04-24     uintah     UINTAH   
1969-03-14  1975-03-20           1975-03-20     summit     SUMMIT   
            1988-04-24           1988-04-24    daggett    DAGGETT   
1976-02-10  NaT                         NaT       juab       JUAB   
1982-02-16  NaT                         NaT    millard    MILLARD   

                                   SHP_ID SHP_FIPS  SHP_VERSION  ...  \
CHANGE_DATE CHANGE_END_DATE                                      ...   
1892-03-09  1896-01-04       uts_boxelder    49003          5.0  ...   
            1896-01-04          uts_cache    49005          3.0  ...   
            1896-01-04         uts_morgan    49029          1.0  ...   
            1896-01-04           uts_rich    49033          4.0  ...   
            1896-01-04          uts_weber    49057          7.0  ...   
1896-01-05  1915-01-03        uts_wasatch    49051          3.0  ...   
            1917-03-04         uts_uintah    49047          2.0  ...   
            NaT                  uts_utah    49049          7.0  ...   
1914-08-13  1915-01-03                NaN      NaN          NaN  ...   
1915-01-04  1917-03-04       uts_duchesne    49013          1.0  ...   
            NaT               uts_wasatch    49051     

In [4]:
#: For each row, assign it to a version (defined by the CHANGE_DATE) if the rows startdate <= CHANGE_DATE <= enddate (or enddate = NaT)
#: One row can belong to multiple versions
#: Thus, we loop through the rows, and for each row we have to loop through the versions to see which it belongs to

def get_unique_district_versions(unique_dates, start_date, end_date):
    versions_list = []
    for date in unique_dates:
        if start_date <= date <= end_date:
            versions_list.append(date)
            continue
        if start_date <= date and pd.isnull(end_date):
            versions_list.append(date)
            
    return versions_list

unique_change_dates = list(dates['CHANGE_DATE'].unique())
# versions_dict = {date : [] for date in unique_change_dates}
versions_dict = {row_key : get_unique_district_versions(unique_change_dates, start_date, end_date) 
                 for row_key, start_date, end_date
                 in zip(dates['UNIQUE_ROW_KEY'], dates['CHANGE_DATE'], dates['CHANGE_END_DATE'])}

versions_dict
versions_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in versions_dict.items()]))
versions_df

,uts_boxelder_S5__box elder_D4,uts_cache_S3__cache_D4,uts_morgan_S1__morgan_D3,uts_rich_S4__rich_D3,uts_weber_S7__weber_D5,uts_wasatch_S3__wasatch_D2,uts_uintah_S2__uintah_D2,uts_utah_S7__utah_D5,nan__duchesne_D1,uts_duchesne_S1__duchesne_D1,uts_wasatch_S4__wasatch_D2,uts_uintah_S3__uintah_D2,uts_duchesne_S2__duchesne_D1,uts_uintah_S4__uintah_D2,uts_uintah_S5__uintah_D2,uts_summit_S10__summit_D3,uts_daggett_S2__daggett_D3,uts_juab_S11__juab_D4,uts_millard_S11__millard_D4
0,1892-03-09,1892-03-09,1892-03-09,1892-03-09,1892-03-09,1896-01-05,1896-01-05,1896-01-05,1914-08-13,1915-01-04,1915-01-04,1917-03-05,1917-03-05,1918-01-07,1919-12-31,1969-03-14,1969-03-14,1976-02-10,1982-02-16
1,NaT,NaT,NaT,NaT,NaT,1914-08-13,1914-08-13,1914-08-13,NaT,NaT,1917-03-05,NaT,1918-01-07,NaT,1969-03-14,NaT,1976-02-10,1982-02-16,NaT
2,NaT,NaT,NaT,NaT,NaT,NaT,1915-01-04,1915-01-04,NaT,NaT,1918-01-07,NaT,1919-12-31,NaT,1976-02-10,NaT,1982-02-16,NaT,NaT
3,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1917-03-05,NaT,NaT,1919-12-31,NaT,1969-03-14,NaT,1982-02-16,NaT,NaT,NaT,NaT
4,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1918-01-07,NaT,NaT,1969-03-14,NaT,1976-02-10,NaT,NaT,NaT,NaT,NaT,NaT
5,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1919-12-31,NaT,NaT,1976-02-10,NaT,1982-02-16,NaT,NaT,NaT,NaT,NaT,NaT
6,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1969-03-14,NaT,NaT,1982-02-16,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1976-02-10,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1982-02-16,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [37]:
all_versions_df = pd.read_pickle(r'C:\gis\Projects\HistoricCounties\Data\JudicialDistricts\district_versions.pkl')
all_versions_df.set_index(['DST_VERSION_KEY', 'UNIQUE_ROW_KEY'], drop=False, inplace=True)
all_versions_df.sort_index(inplace=True)
all_versions_df

UNIQUE_ROW_KEY  \
DST_VERSION_KEY UNIQUE_ROW_KEY                                                 
1N_1880-01-20   uts_morgan_S1__morgan_D2            uts_morgan_S1__morgan_D2   
1N_1880-02-20   uts_boxelder_S5__box elder_D3  uts_boxelder_S5__box elder_D3   
                uts_cache_S3__cache_D3                uts_cache_S3__cache_D3   
                uts_morgan_S1__morgan_D2            uts_morgan_S1__morgan_D2   
                uts_rich_S4__rich_D2                    uts_rich_S4__rich_D2   
...                                                                      ...   
n/a_1865-01-16  uts_piute_S1__nan                          uts_piute_S1__nan   
                uts_rich_S1__nan                            uts_rich_S1__nan   
                uts_sevier_S1__nan                        uts_sevier_S1__nan   
n/a_1882-03-09  uts_garfield_S1__nan                    uts_garfield_S1__nan   
n/a_1884-03-13  uts_garfield_S2__nan                    uts_garfield_S2__nan   

                                              DST_VERSION_KEY CHANGE_DATE  \
DST_VERSION_KEY UNIQUE_ROW_KEY                                              
1N_1880-01-20   uts_morgan_S1__morgan_D2        1N_1880-01-20  1880-01-20   
1N_1880-02-20   uts_boxelder_S5__box elder_D3   1N_1880-02-20  1880-02-20   
                uts_cache_S3__cache_D3          1N_1880-02-20  1880-02-20   
                uts_morgan_S1__morgan_D2        1N_1880-02-20  1880-01-20   
                uts_rich_S4__rich_D2            1N_1880-02-20  1880-02-20   
...                                                       ...         ...   
n/a_1865-01-16  uts_piute_S1__nan              n/a_1865-01-16  1865-01-16   
                uts_rich_S1__nan               n/a_1865-01-16  1864-01-16   
                uts_sevier_S1__nan             n/a_1865-01-16  1865-01-16   
n/a_1882-03-09  uts_garfield_S1__nan           n/a_1882-03-09  1882-03-09   
n/a_1884-03-13  uts_garfield_S2__nan           n/a_1884-03-13  1884-03-13   

                                              COUNTY_KEY   county_version  \
DST_VERSION_KEY UNIQUE_ROW_KEY                                              
1N_1880-01-20   uts_morgan_S1__morgan_D2          morgan    uts_morgan_S1   
1N_1880-02-20   uts_boxelder_S5__box elder_D3   boxelder  uts_boxelder_S5   
                uts_cache_S3__cache_D3             cache     uts_cache_S3   
                uts_morgan_S1__morgan_D2          morgan    uts_morgan_S1   
                uts_rich_S4__rich_D2                rich      uts_rich_S4   
...                                                  ...              ...   
n/a_1865-01-16  uts_piute_S1__nan                  piute     uts_piute_S1   
                uts_rich_S1__nan                    rich      uts_rich_S1   
                uts_sevier_S1__nan                sevier    uts_sevier_S1   
n/a_1882-03-09  uts_garfield_S1__nan            garfield  uts_garfield_S1   
n/a_1884-03-13  uts_garfield_S2__nan            garfield  uts_garfield_S2   

                                              DST_NUMBER district_version  \
DST_VERSION_KEY UNIQUE_ROW_KEY                                              
1N_1880-01-20   uts_morgan_S1__morgan_D2              1N        morgan_D2   
1N_1880-02-20   uts_boxelder_S5__box elder_D3         1N     box elder_D3   
                uts_cache_S3__cache_D3                1N         cache_D3   
                uts_morgan_S1__morgan_D2              1N        morgan_D2   
                uts_rich_S4__rich_D2                  1N          rich_D2   
...                                                  ...              ...   
n/a_1865-01-16  uts_piute_S1__nan                    n/a              n/a   
                uts_rich_S1__nan                     n/a              n/a   
                uts_sevier_S1__nan                   n/a              n/a   
n/a_1882-03-09  uts_garfield_S1__nan                 n/a              n/a   
n/a_1884-03-13  uts_garfield_S2__nan                 n/a              n/a   

               

In [27]:
#: separate dict items with date as key
#: Obsolete

district = '4'
versions = defaultdict(list)
for row_key, version_list in versions_dict.items():
    for date in version_list:
        #: date: [(row_key1, d_key1), (row_key2, d_key1)]
        versions[date].append((row_key, f'{district}_{pd.to_datetime(date).strftime("%Y-%m-%d")}'))
        
version_dfs = defaultdict(pd.DataFrame)
for date in versions:
    version_dfs[date] = pd.DataFrame(versions[date], columns=['unique_row_key', 'district_key'])
# versions_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in versions.items()]))
# versions_df
for date, df in version_dfs.items():
    print(date)
    print(df)
    print()

1892-03-09T00:00:00.000000000
                  unique_row_key  district_key
0  uts_boxelder_S5__box elder_D4  4_1892-03-09
1         uts_cache_S3__cache_D4  4_1892-03-09
2       uts_morgan_S1__morgan_D3  4_1892-03-09
3           uts_rich_S4__rich_D3  4_1892-03-09
4         uts_weber_S7__weber_D5  4_1892-03-09

1896-01-05T00:00:00.000000000
               unique_row_key  district_key
0  uts_wasatch_S3__wasatch_D2  4_1896-01-05
1    uts_uintah_S2__uintah_D2  4_1896-01-05
2        uts_utah_S7__utah_D5  4_1896-01-05

1914-08-13T00:00:00.000000000
               unique_row_key  district_key
0  uts_wasatch_S3__wasatch_D2  4_1914-08-13
1    uts_uintah_S2__uintah_D2  4_1914-08-13
2        uts_utah_S7__utah_D5  4_1914-08-13
3            nan__duchesne_D1  4_1914-08-13

1915-01-04T00:00:00.000000000
                 unique_row_key  district_key
0      uts_uintah_S2__uintah_D2  4_1915-01-04
1          uts_utah_S7__utah_D5  4_1915-01-04
2  uts_duchesne_S1__duchesne_D1  4_1915-01-04
3    uts_wasatc

In [29]:
#: one unified dataframe with district key

district = '4'
versions = []
for row_key, version_list in versions_dict.items():
    for date in version_list:
        #: date: [(row_key1, d_key1), (row_key2, d_key1)]
        versions.append((row_key, f'{district}_{pd.to_datetime(date).strftime("%Y-%m-%d")}'))
        
version_df = pd.DataFrame(versions, columns=['unique_row_key', 'district_key'])
version_df

,unique_row_key,district_key
0,uts_boxelder_S5__box elder_D4,4_1892-03-09
1,uts_cache_S3__cache_D4,4_1892-03-09
2,uts_morgan_S1__morgan_D3,4_1892-03-09
3,uts_rich_S4__rich_D3,4_1892-03-09
4,uts_weber_S7__weber_D5,4_1892-03-09
5,uts_wasatch_S3__wasatch_D2,4_1896-01-05
6,uts_wasatch_S3__wasatch_D2,4_1914-08-13
7,uts_uintah_S2__uintah_D2,4_1896-01-05
8,uts_uintah_S2__uintah_D2,4_1914-08-13
9,uts_uintah_S2__uintah_D2,4_1915-01-04


# Dissolving 2 troubleshooting

In [3]:
all_versions_df = pd.read_pickle(r'C:\gis\Projects\HistoricCounties\Data\JudicialDistricts\district_versions.pkl')
# all_versions_df.to_csv(r'C:\gis\Projects\HistoricCounties\Data\JudicialDistricts\district_versions.csv')

In [8]:
all_versions_df[all_versions_df['UNIQUE_ROW_KEY'] == 'uts_wayne_S1__wayne_D2']

,UNIQUE_ROW_KEY,DST_VERSION_KEY,CHANGE_DATE,COUNTY_KEY,county_version,DST_NUMBER,district_version,FID,Shape,SHP_NAME,...,DST_NAME,DST_END_DATE,NewDistrict,OldDistrict,Version,county_key_y,DST_KEY,DST_START_DATE,COMBINED_DST_KEY,CHANGE_END_DATE
69,uts_wayne_S1__wayne_D2,6_1896-01-06,1896-01-06,wayne,uts_wayne_S1,6,wayne_D2,183.0,"(-110.90366246377673, 38.324363612104015)",WAYNE,...,Wayne,NaT,6,2,2.0,wayne,wayne_D2,1896-01-06,6_1896-01-06,NaT
70,uts_wayne_S1__wayne_D2,6_1917-11-16,1896-01-06,wayne,uts_wayne_S1,6,wayne_D2,183.0,"(-110.90366246377673, 38.324363612104015)",WAYNE,...,Wayne,NaT,6,2,2.0,wayne,wayne_D2,1896-01-06,6_1896-01-06,NaT
71,uts_wayne_S1__wayne_D2,6_1918-01-07,1896-01-06,wayne,uts_wayne_S1,6,wayne_D2,183.0,"(-110.90366246377673, 38.324363612104015)",WAYNE,...,Wayne,NaT,6,2,2.0,wayne,wayne_D2,1896-01-06,6_1896-01-06,NaT
72,uts_wayne_S1__wayne_D2,6_1919-03-13,1896-01-06,wayne,uts_wayne_S1,6,wayne_D2,183.0,"(-110.90366246377673, 38.324363612104015)",WAYNE,...,Wayne,NaT,6,2,2.0,wayne,wayne_D2,1896-01-06,6_1896-01-06,NaT
73,uts_wayne_S1__wayne_D2,6_1919-12-31,1896-01-06,wayne,uts_wayne_S1,6,wayne_D2,183.0,"(-110.90366246377673, 38.324363612104015)",WAYNE,...,Wayne,NaT,6,2,2.0,wayne,wayne_D2,1896-01-06,6_1896-01-06,NaT
74,uts_wayne_S1__wayne_D2,6_1969-03-14,1896-01-06,wayne,uts_wayne_S1,6,wayne_D2,183.0,"(-110.90366246377673, 38.324363612104015)",WAYNE,...,Wayne,NaT,6,2,2.0,wayne,wayne_D2,1896-01-06,6_1896-01-06,NaT
75,uts_wayne_S1__wayne_D2,6_1988-04-25,1896-01-06,wayne,uts_wayne_S1,6,wayne_D2,183.0,"(-110.90366246377673, 38.324363612104015)",WAYNE,...,Wayne,NaT,6,2,2.0,wayne,wayne_D2,1896-01-06,6_1896-01-06,NaT
76,uts_wayne_S1__wayne_D2,6_1914-08-13,1896-01-06,wayne,uts_wayne_S1,6,wayne_D2,183.0,"(-110.90366246377673, 38.324363612104015)",WAYNE,...,Wayne,NaT,6,2,2.0,wayne,wayne_D2,1896-01-06,6_1896-01-06,NaT
77,uts_wayne_S1__wayne_D2,6_1915-01-04,1896-01-06,wayne,uts_wayne_S1,6,wayne_D2,183.0,"(-110.90366246377673, 38.324363612104015)",WAYNE,...,Wayne,NaT,6,2,2.0,wayne,wayne_D2,1896-01-06,6_1896-01-06,NaT
78,uts_wayne_S1__wayne_D2,6_1917-03-05,1896-01-06,wayne,uts_wayne_S1,6,wayne_D2,183.0,"(-110.90366246377673, 38.324363612104015)",WAYNE,...,Wayne,NaT,6,2,2.0,wayne,wayne_D2,1896-01-06,6_1896-01-06,NaT


In [16]:
pd.options.display.max_rows = None
six = all_versions_df[all_versions_df['DST_NUMBER'] == '6'].copy()
sorted(list(six['CHANGE_DATE'].unique()))

[numpy.datetime64('1896-01-06T00:00:00.000000000'),
 numpy.datetime64('1921-03-10T00:00:00.000000000'),
 numpy.datetime64('1931-03-23T00:00:00.000000000'),
 numpy.datetime64('1969-03-14T00:00:00.000000000'),
 numpy.datetime64('1982-02-16T00:00:00.000000000')]

In [19]:
six.set_index(['UNIQUE_ROW_KEY', 'DST_VERSION_KEY'], drop=False, inplace=True)
six.sort_index()

UNIQUE_ROW_KEY  \
UNIQUE_ROW_KEY               DST_VERSION_KEY                                 
uts_garfield_S2__garfield_D3 6_1982-02-16     uts_garfield_S2__garfield_D3   
                             6_1988-04-25     uts_garfield_S2__garfield_D3   
                             6_2003-01-01     uts_garfield_S2__garfield_D3   
uts_kane_S4__kane_D2         6_1896-01-06             uts_kane_S4__kane_D2   
                             6_1913-03-19             uts_kane_S4__kane_D2   
                             6_1914-08-13             uts_kane_S4__kane_D2   
                             6_1915-01-04             uts_kane_S4__kane_D2   
                             6_1917-03-05             uts_kane_S4__kane_D2   
                             6_1917-05-08             uts_kane_S4__kane_D2   
                             6_1917-11-16             uts_kane_S4__kane_D2   
                             6_1918-01-07             uts_kane_S4__kane_D2   
                             6_1919-03-08             uts_kane_S4__kane_D2   
                             6_1919-03-13             uts_kane_S4__kane_D2   
                             6_1919-12-31             uts_kane_S4__kane_D2   
                             6_1921-01-19             uts_kane_S4__kane_D2   
                             6_1921-03-10             uts_kane_S4__kane_D2   
                             6_1931-03-23             uts_kane_S4__kane_D2   
                             6_1969-03-14             uts_kane_S4__kane_D2   
                             6_1975-03-21             uts_kane_S4__kane_D2   
                             6_1976-02-10             uts_kane_S4__kane_D2   
                             6_1982-02-16             uts_kane_S4__kane_D2   
                             6_1988-04-25             uts_kane_S4__kane_D2   
                             6_2003-01-01             uts_kane_S4__kane_D2   
uts_piute_S5__piute_D3       6_1896-01-06           uts_piute_S5__piute_D3   
                             6_1913-03-19           uts_piute_S5__piute_D3   
                             6_1914-08-13           uts_piute_S5__piute_D3   
                             6_1915-01-04           uts_piute_S5__piute_D3   
                             6_1917-03-05           uts_piute_S5__piute_D3   
                             6_1917-05-08           uts_piute_S5__piute_D3   
                             6_1917-11-16           uts_piute_S5__piute_D3   
                             6_1918-01-07           uts_piute_S5__piute_D3   
                             6_1919-03-08           uts_piute_S5__piute_D3   
                             6_1919-03-13           uts_piute_S5__piute_D3   
                             6_1919-12-31           uts_piute_S5__piute_D3   
                             6_1921-01-19           uts_piute_S5__piute_D3   
                             6_1921-03-10           uts_piute_S5__piute_D3   
uts_piute_S6__piute_D3       6_1931-03-23           uts_piute_S6__piute_D3   
                             6_1969-03-14           uts_piute_S6__piute_D3   
                             6_1975-03-21           uts_piute_S6__piute_D3   
                             6_1976-02-10           uts_piute_S6__piute_D3   
                             6_1982-02-16           uts_piute_S6__piute_D3   
                             6_1988-04-25           uts_piute_S6__piute_D3   
                             6_2003-01-01           uts_piute_S6__piute_D3   
uts_sanpete_S16__sanpete_D5  6_1969-03-14      uts_sanpete_S16__sanpete_D5   
                             6_1975-03-21      uts_sanpete_S16__sanpete_D5   
                             6_1976-02-10      uts_sanpete_S16__sanpete_D5   
                             6_1982-02-16      uts_sanpete_S16__sanpete_D5   
                             6_1988-04-25      uts_sanpete_S16__sanpete_D5   
                             6_2003-01-01      uts_sanpete_S16__sanpete_D5   
uts_sevier_S5__sevier_D2     6_1896-01-06         uts_sevier_S5__sevier_D2   
    